In [332]:
import json
import os, sys
from sys import modules
import _jsonnet
from tqdm.notebook import tqdm
import spacy
from nltk.translate.bleu_score import corpus_bleu
from nltk.tokenize.treebank import TreebankWordDetokenizer

import numpy as np
import random
import importlib
from copy import deepcopy
import editdistance
import datetime

In [16]:
# sys.path.append(os.path.abspath('/Users/mac/Desktop/syt/Deep-Learning/Repos/rat-sql/third_party/wikisql'))
# sys.path.append(os.path.abspath('/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL'))

In [17]:
from ratsql.commands.infer import Inferer
from ratsql.datasets.spider import SpiderItem
from ratsql.utils import registry

In [28]:
import torch
from allennlp.common.util import START_SYMBOL, END_SYMBOL

from SpeakQL.Allennlp_models.utils.spider import process_sql, evaluation
from SpeakQL.Allennlp_models.utils.misc_utils import EvaluateSQL, EvaluateSQL_full, \
    Postprocess_rewrite_seq, Postprocess_rewrite_seq_freeze_POS, Postprocess_rewrite_seq_modify_POS

In [19]:
from SpeakQL.Allennlp_models.utils import misc_utils
importlib.reload(misc_utils)
from SpeakQL.Allennlp_models.utils.misc_utils import EvaluateSQL, EvaluateSQL_full, \
    Postprocess_rewrite_seq, Postprocess_rewrite_seq_freeze_POS, Postprocess_rewrite_seq_modify_POS

In [20]:
nlp = spacy.load('en_core_web_sm')

In [ ]:
# del modules['SpeakQL.Allennlp_models.utils.misc_utils']
# del EvaluateSQL, EvaluateSQL_full, Postprocess_rewrite_seq
# from SpeakQL.Allennlp_models.utils.misc_utils import EvaluateSQL, EvaluateSQL_full, Postprocess_rewrite_seq

### Load Rat-sql

In [22]:
def Load_Rat_sql(root_dir,
                 exp_config_path,
                 model_dir,
                 checkpoint_step=40000):

    exp_config = json.loads(_jsonnet.evaluate_file(exp_config_path))
    
    model_config_path = os.path.join(root_dir, exp_config["model_config"])
    model_config_args = exp_config.get("model_config_args")
    
    infer_config = json.loads(_jsonnet.evaluate_file(model_config_path, tla_codes={'args': json.dumps(model_config_args)}))

    inferer = Inferer(infer_config)
    inferer.device = torch.device("cpu")
    model = inferer.load_model(model_dir, checkpoint_step)
    dataset = registry.construct('dataset', inferer.config['data']['val'])

    for _, schema in dataset.schemas.items():
        model.preproc.enc_preproc._preprocess_schema(schema)
    
    _ret_dict = {
        'model': model,
        'dataset': dataset,
        'inferer': inferer,
    }
    
    return _ret_dict
    

In [23]:
rat_sql_model_dict = Load_Rat_sql(root_dir='/Users/mac/Desktop/syt/Deep-Learning/Repos/rat-sql',
                                  exp_config_path='/Users/mac/Desktop/syt/Deep-Learning/Repos/rat-sql/experiments/spider-glove-run.jsonnet',
                                  model_dir='/Users/mac/Desktop/syt/Deep-Learning/Repos/rat-sql/logdir/glove_run/bs=20,lr=7.4e-04,end_lr=0e0,att=0',
                                  checkpoint_step=40000)
rat_sql_model_dict.keys()


WARNING <class 'ratsql.models.enc_dec.EncDecModel.Preproc'>: superfluous {'name': 'EncDec'}
WARNING <class 'ratsql.models.enc_dec.EncDecModel'>: superfluous {'decoder_preproc': {'grammar': {'clause_order': None, 'end_with_from': True, 'factorize_sketch': 2, 'include_literals': False, 'infer_from_conditions': True, 'name': 'spider', 'output_from': True, 'use_table_pointer': True}, 'max_count': 5000, 'min_freq': 4, 'save_path': 'data/spider/nl2code-glove,cv_link=true', 'use_seq_elem_rules': True}, 'encoder_preproc': {'compute_cv_link': True, 'compute_sc_link': True, 'count_tokens_in_word_emb_for_vocab': True, 'db_path': 'data/spider/database', 'fix_issue_16_primary_keys': True, 'include_table_name_in_column': False, 'max_count': 5000, 'min_freq': 4, 'save_path': 'data/spider/nl2code-glove,cv_link=true', 'word_emb': {'kind': '42B', 'lemmatize': True, 'name': 'glove'}}}


Loading model from /Users/mac/Desktop/syt/Deep-Learning/Repos/rat-sql/logdir/glove_run/bs=20,lr=7.4e-04,end_lr=0e0,att=0/model_checkpoint-00040000


DB connections: 100%|██████████| 166/166 [00:02<00:00, 79.09it/s]


dict_keys(['model', 'dataset', 'inferer'])

In [ ]:
rat_sql_asr_model_dict = Load_Rat_sql(root_dir='/Users/mac/Desktop/syt/Deep-Learning/Repos/rat-sql',
                                      exp_config_path='/Users/mac/Desktop/syt/Deep-Learning/Repos/rat-sql/experiments/spider-glove-ASR-run.jsonnet',
                                      model_dir='/Users/mac/Desktop/syt/Deep-Learning/Repos/rat-sql/logdir/glove_ASR_run/ASR,bs=20,lr=7.4e-04,end_lr=0e0,att=0',
                                      checkpoint_step=40000)
rat_sql_asr_model_dict.keys()


In [ ]:
rat_sql_mixed_model_dict = Load_Rat_sql(root_dir='/Users/mac/Desktop/syt/Deep-Learning/Repos/rat-sql',
                                        exp_config_path='/Users/mac/Desktop/syt/Deep-Learning/Repos/rat-sql/experiments/spider-glove-mixed-run.jsonnet',
                                        model_dir='/Users/mac/Desktop/syt/Deep-Learning/Repos/rat-sql/logdir/glove_mixed_run/mixed,bs=20,lr=7.4e-04,end_lr=0e0,att=0',
                                        checkpoint_step=40000)
rat_sql_mixed_model_dict.keys()

In [24]:
model_dicts = {
    'orig': rat_sql_model_dict,
#     'asr': rat_sql_asr_model_dict,
#     'mixed': rat_sql_mixed_model_dict,
}

### Functions

In [119]:
def Question(q, db_id, model_dict=model_dicts['orig']):
    model = model_dict['model']
    dataset = model_dict['dataset']
    inferer = model_dict['inferer']
    
    spider_schema = dataset.schemas[db_id]
    data_item = SpiderItem(
        text=None,  # intentionally None -- should be ignored when the tokenizer is set correctly
        code=None,
        schema=spider_schema,
        orig_schema=spider_schema.orig,
        orig={"question": q}
    )
    model.preproc.clear_items()
    enc_input = model.preproc.enc_preproc.preprocess_item(data_item, None)
    preproc_data = enc_input, None
    with torch.no_grad():
        return inferer._infer_one(model, data_item, preproc_data, beam_size=1, use_heuristic=True)

In [120]:
Question("how many singers do we have?", "concert_singer")

[{'orig_question': 'how many singers do we have?',
  'model_output': {'_type': 'sql',
   'select': {'_type': 'select',
    'is_distinct': False,
    'aggs': [{'_type': 'agg',
      'agg_id': {'_type': 'Count'},
      'val_unit': {'_type': 'Column',
       'col_unit1': {'_type': 'col_unit',
        'agg_id': {'_type': 'NoneAggOp'},
        'col_id': 0,
        'is_distinct': False}}}]},
   'sql_where': {'_type': 'sql_where'},
   'sql_groupby': {'_type': 'sql_groupby'},
   'sql_orderby': {'_type': 'sql_orderby', 'limit': False},
   'sql_ieu': {'_type': 'sql_ieu'},
   'from': {'_type': 'from',
    'table_units': [{'_type': 'Table', 'table_id': 1}]}},
  'inferred_code': 'SELECT Count(*) FROM singer',
  'score': -0.00011539317730324683}]

In [121]:
Question("display the employee i D and salary of all employees who report to pye um, first name.", "hr_1")

[]

In [124]:
Question("how many singers have held at least 2 concerts?", "concert_singer")

[{'orig_question': 'how many singers have held at least 2 concerts?',
  'model_output': {'_type': 'sql',
   'select': {'_type': 'select',
    'is_distinct': False,
    'aggs': [{'_type': 'agg',
      'agg_id': {'_type': 'Count'},
      'val_unit': {'_type': 'Column',
       'col_unit1': {'_type': 'col_unit',
        'agg_id': {'_type': 'NoneAggOp'},
        'col_id': 0,
        'is_distinct': False}}}]},
   'sql_where': {'_type': 'sql_where'},
   'sql_groupby': {'_type': 'sql_groupby'},
   'sql_orderby': {'_type': 'sql_orderby', 'limit': False},
   'sql_ieu': {'_type': 'sql_ieu'},
   'from': {'_type': 'from',
    'table_units': [{'_type': 'Table', 'table_id': 3}]}},
  'inferred_code': 'SELECT Count(*) FROM singer_in_concert',
  'score': -0.5656531027781071}]

In [127]:
Question("how many singers have held at least 2 concerts ?", "concert_singer")

[{'orig_question': 'how many singers have held at least 2 concerts ?',
  'model_output': {'_type': 'sql',
   'select': {'_type': 'select',
    'is_distinct': False,
    'aggs': [{'_type': 'agg',
      'agg_id': {'_type': 'Count'},
      'val_unit': {'_type': 'Column',
       'col_unit1': {'_type': 'col_unit',
        'agg_id': {'_type': 'NoneAggOp'},
        'col_id': 0,
        'is_distinct': False}}}]},
   'sql_where': {'_type': 'sql_where'},
   'sql_groupby': {'_type': 'sql_groupby'},
   'sql_orderby': {'_type': 'sql_orderby', 'limit': False},
   'sql_ieu': {'_type': 'sql_ieu'},
   'from': {'_type': 'from',
    'table_units': [{'_type': 'Table', 'table_id': 3}]}},
  'inferred_code': 'SELECT Count(*) FROM singer_in_concert',
  'score': -0.5656531027781071}]

In [132]:
Question("how many airlines come from from CA?", "flight_2")

[{'orig_question': 'how many airlines come from from CA?',
  'model_output': {'_type': 'sql',
   'select': {'_type': 'select',
    'is_distinct': False,
    'aggs': [{'_type': 'agg',
      'agg_id': {'_type': 'Count'},
      'val_unit': {'_type': 'Column',
       'col_unit1': {'_type': 'col_unit',
        'agg_id': {'_type': 'NoneAggOp'},
        'col_id': 0,
        'is_distinct': False}}}]},
   'sql_where': {'_type': 'sql_where',
    'where': {'_type': 'Eq',
     'val_unit': {'_type': 'Column',
      'col_unit1': {'_type': 'col_unit',
       'agg_id': {'_type': 'NoneAggOp'},
       'col_id': 3,
       'is_distinct': False}},
     'val1': {'_type': 'Terminal'}}},
   'sql_groupby': {'_type': 'sql_groupby'},
   'sql_orderby': {'_type': 'sql_orderby', 'limit': False},
   'sql_ieu': {'_type': 'sql_ieu'},
   'from': {'_type': 'from',
    'table_units': [{'_type': 'Table', 'table_id': 0}]}},
  'inferred_code': "SELECT Count(*) FROM airlines WHERE airlines.Abbreviation = 'terminal'",
  'scor

In [ ]:
Question("how many singers do we have?", "concert_singer", model_dict=model_dicts['mixed'])

In [ ]:
Question("display the employee i D and salary of all employees who report to pye um, first name.", "hr_1",
         model_dict=model_dicts['mixed'])

### Evaluate the trained model

In [ ]:
dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'

In [ ]:
with open(dev_path, 'r') as f:
    dev_dataset = json.load(f)
len(dev_dataset)

In [ ]:
d = dev_dataset[0]
d

In [ ]:
question(d['question'], d['db_id'])

In [ ]:
pred_sql_list = []

for d in tqdm(dev_dataset):
    pred = question(d['question'], d['db_id'])[0]
    pred_sql_list.append(pred['inferred_code'])

len(pred_sql_list)

In [ ]:
dev_pred_path = './output/dev_output.txt'

In [ ]:
# with open(dev_pred_path, 'w') as f:
#     for pred in pred_sql_list:
#         f.write(pred + '\n')

### Predict SQL given transcribed dataset JSON files

In [ ]:
DATASET = 'train'
input_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/{0}/{0}_asr_amazon.json'.format(DATASET)
output_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/{0}/{0}_asr_amazon_RatsqlPredicted.json'.format(DATASET)


In [ ]:
# Paths for human test 
input_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test_yshao_asr_amazon.json'
output_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test_yshao_asr_amazon_RatsqlPredicted.json'


In [ ]:
with open(input_dataset_path, 'r') as f:
    asr_dataset = json.load(f)
len(asr_dataset)

In [ ]:
asr_dataset[0].keys()

In [ ]:
for i, d in tqdm(enumerate(asr_dataset), total=len(asr_dataset)):
    if 'ratsql_pred_sql' in d:
        continue
        
    pred = Question(d['question'], d['db_id'])
    if len(pred) == 0:
        print('{}: question({}, {}) failed'.format(i, d['question'], d['db_id']))
        d['ratsql_pred_sql'] = ''
    else:
        d['ratsql_pred_sql'] = pred[0]['inferred_code']

asr_dataset[0]

In [ ]:
d

In [ ]:
pred = question(d['question'], d['db_id'])
pred

In [ ]:
# with open(output_dataset_path, 'w') as f:
#     json.dump(asr_dataset, f, indent=4)

### Predict & Evaluate SQL given rewriter output / original / ASR cands

In [12]:
EvaluateSQL(pred_str='SELECT * FROM singer WHERE name = "Joe Sharp"',
            gold_str='SELECT * FROM singer WHERE name = "DEF"',
            db='concert_singer'), \
EvaluateSQL(pred_str='SELECT country FROM singer WHERE name = "ABC"',
            gold_str='SELECT country FROM singer WHERE name = "DEF" ORDER BY age LIMIT 1',
            db='concert_singer')

((1, 1.0, 0), (0, 0.5, 1))

In [298]:
test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/test_rewriter.json'
orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'

In [299]:
with open(test_dataset_path, 'r') as f:
    test_dataset_clean = json.load(f)
with open(orig_dev_path, 'r') as f:
    orig_dev_dataset_clean = json.load(f)

len(test_dataset_clean), sum([len(d) for d in test_dataset_clean]), len(orig_dev_dataset_clean)

(547, 3075, 1034)

#### Black box (first cands)

In [58]:
# test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/Assembly_transcribe/test_rewriter.json'
test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/test_rewriter.json'
orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'

In [59]:
# # human test 
# test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test_yshao_rewriter.json'
# orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test.json'

In [60]:
with open(test_dataset_path, 'r') as f:
    test_dataset = json.load(f)
with open(orig_dev_path, 'r') as f:
    orig_dev_dataset = json.load(f)

len(test_dataset), sum([len(d) for d in test_dataset]), len(orig_dev_dataset)

(547, 3075, 1034)

In [61]:
# Just using the 1st ASR candidate, no correction 

ref_list = []
hyp_list = []
wer_numer = 0
wer_denom = 0

for d in tqdm(test_dataset):
    if len(d) == 0:
        continue
        
    c = d[0]
        
    _o_idx = c['original_id']
    o = orig_dev_dataset[_o_idx]
    assert c['gold_question_toks'] == o['question_toks'], (c['gold_question_toks'], o['question_toks'])
    
    _db_id = o['db_id']
    
#     _pred_sql = Question(c['question'], _db_id, model_dict=model_dicts['orig'])[0]['inferred_code']
    
#     _gold_sql = c['query']
#     _exact, _score, _exec = EvaluateSQL(_pred_sql, _gold_sql, _db_id)
    
#     c['pred_sql'] = _pred_sql
#     c['score'] = _score
#     c['exact'] = _exact
#     c['exec'] = _exec

    _question_toks = [_t.lower() for _t in c['question_toks']]
    _gold_question_toks = [_t.lower() for _t in c['gold_question_toks']]
    ref_list.append([_gold_question_toks])
    hyp_list.append(_question_toks)
    wer_numer += editdistance.eval(_gold_question_toks, _question_toks)
    wer_denom += len(_gold_question_toks)


In [62]:
# Only using the 1st candidate to rewrite 
# _avg_1st = sum([d[0]['score'] for d in test_dataset]) / len(test_dataset)
# _avg_exact_1st = sum([d[0]['exact'] for d in test_dataset]) / len(test_dataset)
# _avg_exec_1st = sum([d[0]['exec'] for d in test_dataset]) / len(test_dataset)

# ## Std-dev (1st cand only)
# _std_1st = np.std([d[0]['score'] for d in test_dataset])

## BLEU 
_bleu = corpus_bleu(list_of_references=ref_list,
                    hypotheses=hyp_list)
_wer = 1.0 * wer_numer / (wer_denom + 1e-9)

# print('avg = {:.4f} (std = {:.4f})'.format(_avg_1st, _std_1st))
# print('avg = {:.4f}'.format(_avg_1st))
# print('avg_exact = {:.4f}'.format(_avg_exact_1st))
# print('avg_exec = {:.4f}'.format(_avg_exec_1st))
print(f'BLEU = {_bleu:.4f}')
print(f'WER = {_wer:.4f}')


BLEU = 0.8010
WER = 0.1194


In [63]:
wer_numer, wer_denom

(853, 7146)

In [ ]:
EvaluateSQL_full(plist=[d[0]['pred_sql'] for d in test_dataset],
                 glist=[d[0]['query'] for d in test_dataset],
                 db_id_list=[d[0]['db_id'] for d in test_dataset],
                 etype='all')

In [36]:
# Dump the predictions 
# Orig
# test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-test-save/First-cands.json'
# test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-test-save/Assembly/First-cands.json'
# ASR
# test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-asr-test-save/First-cands.json'
# test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-asr-test-save/Assembly/First-cands.json'
# Mixed
# test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-mixed-test-save/First-cands.json'
# test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-mixed-test-save/Assembly/First-cands.json'

# with open(test_output_path, 'w') as f:
#     json.dump(test_dataset, f, indent=4)

#### Gold (original text)

In [38]:
# Using original text (no ASR)

with open(test_dataset_path, 'r') as f:
    test_dataset = json.load(f)
with open(orig_dev_path, 'r') as f:
    orig_dev_dataset = json.load(f)

len(test_dataset), sum([len(d) for d in test_dataset]), len(orig_dev_dataset)

(547, 3075, 1034)

In [44]:
# Using original text (no ASR)

for d in tqdm(test_dataset):
    if len(d) == 0:
        continue
        
    c = d[0]
        
    _o_idx = c['original_id']
    o = orig_dev_dataset[_o_idx]
    assert c['gold_question_toks'] == o['question_toks'], (c['gold_question_toks'], o['question_toks'])
    
    _db_id = o['db_id']
    
    _pred_sql = Question(c['gold_question'], _db_id, model_dict=model_dicts['orig'])[0]['inferred_code']
    
    _gold_sql = c['query']
    _exact, _score, _exec = EvaluateSQL(_pred_sql, _gold_sql, _db_id)
    
    c['pred_sql'] = _pred_sql
    c['score'] = _score
    c['exact'] = _exact
    c['exec'] = _exec


SELECT Max(performance.Share), Min(performance.Official_ratings_(millions)) FROM performance WHERE performance.Type != 'terminal'
SELECT max(SHARE) ,  min(SHARE) FROM performance WHERE TYPE != "Live final"
orchestra
process_sql.get_sql() failed



In [45]:
# Only using the 1st candidate to rewrite 
_avg_1st = sum([d[0]['score'] for d in test_dataset]) / len(test_dataset)
_avg_exact_1st = sum([d[0]['exact'] for d in test_dataset]) / len(test_dataset)
_avg_exec_1st = sum([d[0]['exec'] for d in test_dataset]) / len(test_dataset)

# ## Std-dev (1st cand only)
# _std_1st = np.std([d[0]['score'] for d in test_dataset])

# print('avg = {:.4f} (std = {:.4f})'.format(_avg_1st, _std_1st))
print('avg = {:.4f}'.format(_avg_1st))
print('avg_exact = {:.4f}'.format(_avg_exact_1st))
print('avg_exec = {:.4f}'.format(_avg_exec_1st))


avg = 0.8316
avg_exact = 0.6234
avg_exec = 0.4095


In [ ]:
EvaluateSQL_full(plist=[d[0]['pred_sql'] for d in test_dataset],
                 glist=[d[0]['query'] for d in test_dataset],
                 db_id_list=[d[0]['db_id'] for d in test_dataset],
                 etype='all')

In [ ]:
random_subset = random.sample(test_dataset, k=100)

EvaluateSQL_full(plist=[d[0]['pred_sql'] for d in random_subset],
                 glist=[d[0]['query'] for d in random_subset],
                 db_id_list=[d[0]['db_id'] for d in random_subset],
                 etype='all')

#### Simple heuristics

In [310]:
def Postprocess_rewrite_seq_heuristics(cand_dict, pred_dict=None):
    '''
    Heuristics: (not using preds)
    [punct] it's [punct] -> ids
    i d -> id
    idea -> id
    odds -> ids
    [punct] id [punct] -> id
    [punct] ids [punct] -> ids
    "Whxx/How": ending "." -> "?"
    number: text -> digits
    coats -> codes
    '''
    
    Num_dict = {
        "one": "1",
        "two": "2",
        "three": "3",
        "four": "4",
        "five": "5",
        "six": "6",
        "seven": "7",
        "eight": "8",
        "nine": "9",
        "ten": "10",
        "eleven": "11",
        "twelve": "12",
        "thirteen": "13",
        "fourteen": "14",
        "fifteen": "15",
        "sixteen": "16",
        "seventeen": "17",
        "eighteen": "18",
        "nineteen": "19",
        "twenty": "20",
    }
    
    question_toks = ["[PAD]"] + [t.lower() for t in cand_dict['question_toks']] + ["[PAD]"]
    rewritten_toks = []
    
    ptr = 1
    while ptr < len(question_toks) - 1:
        _tok = question_toks[ptr]
        _o_tok = _tok
        if _tok in {"it's", "ids", "id"}:
            if _tok == "it's":
                _o_tok = "ids"
            # other (ids, id) unchanged 
            
            # remove surronding puncts
            if question_toks[ptr - 1] in '.,?':
                rewritten_toks = rewritten_toks[:-1]
            if question_toks[ptr + 1] in '.,?':
                ptr += 1
        elif _tok == 'odds':
            _o_tok = 'ids'
        elif _tok == 'idea':
            _o_tok = 'id'
        elif _tok in Num_dict:
            _o_tok = Num_dict[_tok]
        elif _tok == 'coats':
            _o_tok == 'codes'
#         elif ptr == len(question_toks) - 2 and _tok in '.?':
#             if question_toks[1].startswith('how') or question_toks[1].startswith('wh'):
#                 _o_tok = '?'
#             else:
#                 _o_tok = '.'
        rewritten_toks.append(_o_tok)
        ptr += 1
        
    if rewritten_toks[-1] not in '.?':
        rewritten_toks.append('.')
    if question_toks[1].startswith('how') or question_toks[1].startswith('wh'):
        rewritten_toks[-1] = '?'
    else:
        rewritten_toks[-1] = '.'
        
    return rewritten_toks

In [311]:
test_dataset = deepcopy(test_dataset_clean)
orig_dev_dataset = deepcopy(orig_dev_dataset_clean)

len(test_dataset), sum([len(d) for d in test_dataset]), len(orig_dev_dataset)

(547, 3075, 1034)

In [312]:
ref_list = []
hyp_list = []

for d in tqdm(test_dataset):
    if len(d) == 0:
        continue
        
    c = d[0]
        
    _o_idx = c['original_id']
    o = orig_dev_dataset[_o_idx]
    assert c['gold_question_toks'] == o['question_toks'], (c['gold_question_toks'], o['question_toks'])
    
    _db_id = o['db_id']
    
    _rewritten_question_toks = Postprocess_rewrite_seq_heuristics(c, None)
    _rewritten_question = ' '.join(_rewritten_question_toks)
    
    _pred_sql = Question(_rewritten_question, _db_id, model_dict=model_dicts['orig'])[0]['inferred_code']
    
    _gold_sql = c['query']
    _exact, _score, _exec = EvaluateSQL(_pred_sql, _gold_sql, _db_id)
    
    c['pred_sql'] = _pred_sql
    c['score'] = _score
    c['exact'] = _exact
    c['exec'] = _exec
    _gold_question_toks = [_t.lower() for _t in c['gold_question_toks']]
    ref_list.append([_gold_question_toks])
    hyp_list.append(_rewritten_question_toks)


In [313]:
# Only using the 1st candidate to rewrite 
_avg_1st = sum([d[0]['score'] for d in test_dataset]) / len(test_dataset)
_avg_exact_1st = sum([d[0]['exact'] for d in test_dataset]) / len(test_dataset)
_avg_exec_1st = sum([d[0]['exec'] for d in test_dataset]) / len(test_dataset)
_bleu = corpus_bleu(list_of_references=ref_list,
                    hypotheses=hyp_list)
    
# ## Std-dev (1st cand only)
# _std_1st = np.std([d[0]['score'] for d in test_dataset])

# print('avg = {:.4f} (std = {:.4f})'.format(_avg_1st, _std_1st))
print('avg = {:.4f}'.format(_avg_1st))
print('avg_exact = {:.4f}'.format(_avg_exact_1st))
print('avg_exec = {:.4f}'.format(_avg_exec_1st))
print(f'BLEU = {_bleu:.4f}')


avg = 0.7481
avg_exact = 0.4899
avg_exec = 0.3510
BLEU = 0.8306


#### BRIDGE results 

In [72]:
bridge_pred_path = '/Users/mac/Desktop/syt/Deep-Learning/Repos/TabularSemanticParsing/server/spider.bridge.lstm.meta.ts.ppl-0.85.2.dn.eo.feat.bert-base-uncased.xavier-768-400-400-8-4-0.0005-inv-sqr-0.0005-4000-6e-05-inv-sqr-3e-05-4000-0.1-0.3-0.0-0.0-1-8-0.0-0.0-res-0.2-0.0-ff-0.4-0.0.210126-135032.7kf9/predictions.16.ASR.dev.txt'
gold_query_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev_gold.sql'
test_index_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/index_test.txt'
# db_id_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/db_id.txt'

In [73]:
with open(bridge_pred_path, 'r') as f:
    bridge_preds = f.read().strip().split('\n')
with open(gold_query_path, 'r') as f:
    _lines = [l.split('\t') for l in f.read().strip().split('\n')]
    gold_queries = [l[0] for l in _lines]
    db_ids = [l[1] for l in _lines]
# with open(db_id_path, 'r') as f:
#     db_ids = f.read().strip().split('\n')
with open(test_index_path, 'r') as f:
    test_indices = [int(i) for i in f.read().strip().split('\n')]
    
len(bridge_preds), len(gold_queries), len(db_ids), len(test_indices)

(1034, 1034, 1034, 547)

In [74]:
EvaluateSQL_full(plist=[bridge_preds[idx] for idx in test_indices],
                 glist=[gold_queries[idx] for idx in test_indices],
                 db_id_list=[db_ids[idx] for idx in test_indices],
                 etype='all')

eval_err_num:1
eval_err_num:2
eval_err_num:3

                     easy                 medium               hard                 extra                all                 
count                136                  240                  91                   80                   547                 
=====================   EXECUTION ACCURACY     =====================
execution            0.654                0.442                0.451                0.325                0.479               

====================== EXACT MATCHING ACCURACY =====================
exact match          0.750                0.450                0.429                0.287                0.497               

---------------------PARTIAL MATCHING ACCURACY----------------------
select               0.882                0.679                0.824                0.762                0.767               
select(no AGG)       0.904                0.688                0.824                0.762                0.776     

In [75]:
EvaluateSQL_full(plist=bridge_preds,
                 glist=gold_queries,
                 db_id_list=db_ids,
                 etype='all')

eval_err_num:1
eval_err_num:2
eval_err_num:3
eval_err_num:4

                     easy                 medium               hard                 extra                all                 
count                250                  440                  174                  170                  1034                
=====================   EXECUTION ACCURACY     =====================
execution            0.652                0.439                0.529                0.276                0.479               

====================== EXACT MATCHING ACCURACY =====================
exact match          0.748                0.468                0.500                0.253                0.506               

---------------------PARTIAL MATCHING ACCURACY----------------------
select               0.871                0.666                0.851                0.612                0.738               
select(no AGG)       0.900                0.670                0.862                0.612           

### Variations

#### End2end

In [205]:
VERSION = '4.1.0.1_laststep'

end2end_pred_path = '/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/output-{}.json'.format(VERSION)
# test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/test_reranker.json'
orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'


In [206]:
with open(end2end_pred_path, 'r') as f:
    end2end_preds = [json.loads(l) for l in f]
with open(orig_dev_path, 'r') as f:
    orig_dev_dataset = json.load(f)
    
len(end2end_preds), len(orig_dev_dataset)

(3075, 1034)

In [207]:
# Quick evaluation: only using the 1st ASR candidate

_seen_ids = set()
_first_cand_preds = []

for i, p in tqdm(enumerate(end2end_preds), total=len(end2end_preds)):

    # p = rewriter_ILM_preds[pred_idx]
    # _o_idx = c['original_id']
    # o = orig_dev_dataset[_o_idx]
    # assert ' '.join(c['question_toks']) == p['question'], (' '.join(c['question_toks']), p['question'])
    # assert c['gold_question_toks'] == o['question_toks'], (c['gold_question_toks'], o['question_toks'])
    
    _o_idx = p['original_id']
    if _o_idx in _seen_ids:
        continue
    else:
        _seen_ids.add(_o_idx)
        
    o = orig_dev_dataset[_o_idx]
    
    # Debug 
    # assert c['rewriter_tags'] == p['rewriter_tags'][:len(c['rewriter_tags'])], f"{c['rewriter_tags']}\n{p['rewriter_tags']}\nShould raise"

    _db_id = o['db_id']
    _pred_sql = p['pred_sql']
    _gold_sql = p['gold_sql']
    assert _gold_sql == o['query'], (_gold_sql, o['query'])
    
    _exact, _score, _exec = EvaluateSQL(_pred_sql, _gold_sql, _db_id)
    
    # Save prediction results 
    p['score'] = _score
    p['exact'] = _exact
    p['exec'] = _exec
    
    _first_cand_preds.append(p)
    

SELECT Max(performance.Official_ratings_(millions)), Min(performance.Share) FROM performance WHERE performance.Type != 'terminal'
SELECT max(SHARE) ,  min(SHARE) FROM performance WHERE TYPE != "Live final"
orchestra
process_sql.get_sql() failed
SELECT Max(performance.Official_ratings_(millions)), Min(performance.Share) FROM performance WHERE performance.Type != 'terminal'
SELECT max(SHARE) ,  min(SHARE) FROM performance WHERE TYPE != "Live final"
orchestra
process_sql.get_sql() failed



In [208]:
# Only using the 1st candidate to rewrite 
_avg_1st = sum([p['score'] for p in _first_cand_preds]) / len(_first_cand_preds)
_avg_exact_1st = sum([p['exact'] for p in _first_cand_preds]) / len(_first_cand_preds)
_avg_exec_1st = sum([p['exec'] for p in _first_cand_preds]) / len(_first_cand_preds)

## Std-dev (1st cand only)
# _std_1st = np.std([c['score'] for d in test_dataset for c in d if c['is_reranker_selection']])

print('avg = {:.4f}'.format(_avg_1st))
print('avg_exact = {:.4f}'.format(_avg_exact_1st))
print('avg_exec = {:.4f}'.format(_avg_exec_1st))


avg = 0.7660
avg_exact = 0.4973
avg_exec = 0.3620


#### Reranker

In [91]:
VERSION = '1.14.1.2'
HUMAN_TEST = False

if not HUMAN_TEST:
    reranker_pred_path = '/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/output-{}.json'.format(VERSION)
    test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/test_reranker.json'
    orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'
else:
    reranker_pred_path = '/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/output-humantest-yshao-{}.json'.format(VERSION)
    test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test_yshao_reranker.json'
    orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test.json'


In [ ]:
## Aggreg with rewriter cands 

RERANKER_VERSION = '1.10.0.2'
REWRITER_VERSION = '2.6.0.2t-2.6.0.2i'
HUMAN_TEST = True

if not HUMAN_TEST:
    reranker_pred_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/aggreg/output-{RERANKER_VERSION}-with-{REWRITER_VERSION}.json'
    test_dataset_path = f'/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/aggreg_extra_cands/test_reranker_with_{REWRITER_VERSION}.json'
    orig_dev_path = f'/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'
else:
    reranker_pred_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/aggreg/output-humantest-yshao-{RERANKER_VERSION}-with-{REWRITER_VERSION}.json'
    test_dataset_path = f'/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/aggreg_extra_cands/human_test_yshao_reranker_with_{REWRITER_VERSION}.json'
    orig_dev_path = f'/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test.json'
    

In [92]:
with open(reranker_pred_path, 'r') as f:
    reranker_preds = [json.loads(l) for l in f.readlines()]
with open(test_dataset_path, 'r') as f:
    test_dataset = json.load(f)
with open(orig_dev_path, 'r') as f:
    orig_dev_dataset = json.load(f)

len(reranker_preds), len(test_dataset), sum([len(d) for d in test_dataset]), len(orig_dev_dataset)

(3075, 547, 3075, 1034)

In [93]:
reranker_preds[0].keys(), test_dataset[0][0].keys()

(dict_keys(['score_preds', 'question', 'original_id']),
 dict_keys(['db_id', 'query', 'query_toks', 'query_toks_no_value', 'question', 'question_toks', 'sql', 'span_ranges', 'original_id', 'ratsql_pred_sql', 'gold_question', 'gold_question_toks', 'ratsql_pred_exact', 'ratsql_pred_score', 'question_toks_edit_distance']))

In [94]:
pred_idx = 0

ref_list = []
hyp_list = []

for d in tqdm(test_dataset):
    if len(d) == 0:
        continue
    
    # Find the cand with highest score 
    d_preds = []
    for _ in d:
        p = reranker_preds[pred_idx]
        d_preds.append(p)
        pred_idx += 1
    
    _c_idx = np.argmax([p['score_preds'] for p in d_preds])
    
    c = d[_c_idx]
    p = d_preds[_c_idx]
    
    for _c in d:
        _c['is_reranker_selection'] = False
    c['is_reranker_selection'] = True
    
    # Use the selected cand to proceed 
    _o_idx = c['original_id']
    o = orig_dev_dataset[_o_idx]
    assert ' '.join(c['question_toks']) == p['question'], (' '.join(c['question_toks']), p['question'])
    assert c['gold_question_toks'] == o['question_toks'], (c['gold_question_toks'], o['question_toks'])
    
    _db_id = o['db_id']

    _question = c['question']

    if c['ratsql_pred_sql'] is not None:
        # If SQL is already predicted, no need to predict again 
        _pred_sql = c['ratsql_pred_sql']
    else:
        # If SQL not yet predicted, do it 
        _pred_sql = Question(_question, _db_id)[0]['inferred_code']

    _gold_sql = c['query']
    _exact, _score, _exec = EvaluateSQL(_pred_sql, _gold_sql, _db_id)

#     # Save the taggerILM raw outputs, for later aggregation 
#     c['pred_tags'] = p['rewriter_tags']
#     c['pred_ILM'] = p['rewrite_seq_prediction']
#     c['pred_ILM_cands'] = p['rewrite_seq_prediction_cands']
    
    # Save prediction results 
    # c['rewritten_question'] = p['rewritten_question'] = _rewritten_question
    c['pred_sql'] = p['pred_sql'] = _pred_sql
    # p['gold_sql'] = _gold_sql
    c['score'] = p['score'] = _score
    c['exact'] = p['exact'] = _exact
    c['exec'] = p['exec'] = _exec

    _question_toks = [_t.lower() for _t in c['question_toks']]
    _gold_question_toks = [_t.lower() for _t in c['gold_question_toks']]

    ref_list.append([_gold_question_toks])
    hyp_list.append(_question_toks)


In [95]:
# Only using the 1st candidate to rewrite 
_avg_1st = sum([c['score'] for d in test_dataset for c in d if c['is_reranker_selection']]) / len(test_dataset)
_avg_exact_1st = sum([c['exact'] for d in test_dataset for c in d if c['is_reranker_selection']]) / len(test_dataset)
_avg_exec_1st = sum([c['exec'] for d in test_dataset for c in d if c['is_reranker_selection']]) / len(test_dataset)

## Std-dev (1st cand only)
_std_1st = np.std([c['score'] for d in test_dataset for c in d if c['is_reranker_selection']])

## BLEU 
_bleu = corpus_bleu(list_of_references=ref_list,
                    hypotheses=hyp_list)

print('avg = {:.4f} (std = {:.4f})'.format(_avg_1st, _std_1st))
print('avg_exact = {:.4f}'.format(_avg_exact_1st))
print('avg_exec = {:.4f}'.format(_avg_exec_1st))
print(f'BLEU = {_bleu:.4f}')



avg = 0.7599 (std = 0.2977)
avg_exact = 0.5027
avg_exec = 0.3620
BLEU = 0.7853


In [ ]:
EvaluateSQL_full(plist=[c['pred_sql'] for d in test_dataset for c in d if c['is_reranker_selection']],
                 glist=[d[0]['query'] for d in test_dataset],
                 db_id_list=[d[0]['db_id'] for d in test_dataset])

In [96]:
# Non ASR
if not HUMAN_TEST:
    test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-test-save/{VERSION}.json'
else:
    test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-test-save/humantest-yshao-{VERSION}.json'
    
# ASR
# test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-asr-test-save/{VERSION}.json'

with open(test_output_path, 'w') as f:
    json.dump(test_dataset, f, indent=4)

#### Tagger-ILM

In [214]:
## Tagger accuracy 
VERSION_LIST = [f'2.16.0.{v}t' for v in range(5)]

for VERSION in VERSION_LIST:
    predict_fname = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/output-{VERSION}.json'

    with open(predict_fname, 'r') as f:
        predicts = [json.loads(l) for l in f]
    len(predicts)

    correct_tags = 0
    total_tags = 0

    non_O_matches = 0
    non_O_preds = 0
    non_O_golds = 0

    for p in predicts:
        _pred_tags = p['tags_prediction']
        _gold_tags = [t for t in p['gold_tags'] if t != 'O']
        assert len(_pred_tags) == len(_gold_tags)

        _cor = sum([_p == _g for _p, _g in zip(_pred_tags, _gold_tags)])
        _total = len(_pred_tags)
        assert -1e-8 < _cor / _total - p['tags_accuracy'] < 1e-8, f"{_cor / _total}, {p['tags_accuracy']}"

        for _p, _g in zip(_pred_tags, _gold_tags):
            if _p != 'O-KEEP':
                non_O_preds += 1
            if _g != 'O-KEEP':
                non_O_golds += 1
            if _p == _g and _p != 'O-KEEP':
                non_O_matches += 1

        correct_tags += _cor
        total_tags += _total

    tagger_prec = non_O_matches / non_O_preds
    tagger_recall = non_O_matches / non_O_golds
    tagger_F1 = 2 * tagger_prec * tagger_recall / (tagger_prec + tagger_recall)

    print(f'VERSION: {VERSION}')
    print(f'Overall tagger accuracy = {correct_tags}/{total_tags} = {correct_tags / total_tags:.4f}')
    print(f'Tagger P = {tagger_prec:.4f}, R = {tagger_recall:.4f}, F1 = {tagger_F1:.4f}')
    print()
    

VERSION: 2.16.0.0t
Overall tagger accuracy = 38814/41776 = 0.9291
Tagger P = 0.7764, R = 0.6817, F1 = 0.7260

VERSION: 2.16.0.1t
Overall tagger accuracy = 38782/41776 = 0.9283
Tagger P = 0.7529, R = 0.6967, F1 = 0.7237

VERSION: 2.16.0.2t
Overall tagger accuracy = 38742/41776 = 0.9274
Tagger P = 0.7446, R = 0.6858, F1 = 0.7140

VERSION: 2.16.0.3t
Overall tagger accuracy = 38893/41776 = 0.9310
Tagger P = 0.7663, R = 0.6974, F1 = 0.7303

VERSION: 2.16.0.4t
Overall tagger accuracy = 38932/41776 = 0.9319
Tagger P = 0.7640, R = 0.7072, F1 = 0.7345



In [45]:
def _Postprocess_rewrite_seq_wrapper(cand_dict, pred_dict):
    _tags = pred_dict['rewriter_tags']
    _rewrite_seq = pred_dict['rewrite_seq_prediction']
    _question_toks = cand_dict['question_toks']
    return Postprocess_rewrite_seq(_tags, _rewrite_seq, _question_toks)

In [262]:
def Full_evaluate_ILM(eval_version,
                      rewriter_ILM_pred_path,
                      test_dataset_path,
                      orig_dev_path,
                      test_output_path=None,
                      rat_sql_run='orig',
                      ILM_rewrite_func=_Postprocess_rewrite_seq_wrapper):
    
    '''
    eval_version: simply for printing results 
    
    Example paths:
    rewriter_ILM_pred_path = '/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/output-{}.json'.format(VERSION)
    test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/test_rewriter.json'
    orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'
    
    ILM_rewrite_func: Callable, args: (_tags, _rewrite_seq, _question_toks)
    '''
    
    VERSION = eval_version
    
    with open(rewriter_ILM_pred_path, 'r') as f:
        rewriter_ILM_preds = [json.loads(l) for l in f.readlines()]
    with open(test_dataset_path, 'r') as f:
        test_dataset = json.load(f)
    with open(orig_dev_path, 'r') as f:
        orig_dev_dataset = json.load(f)

    # len(rewriter_ILM_preds), len(test_dataset), sum([len(d) for d in test_dataset]), len(orig_dev_dataset)

    ## Quick evaluation: only using the 1st ASR candidate

    pred_idx = 0

    ref_list = []
    hyp_list = []

    for d in tqdm(test_dataset, desc=f'VERSION {VERSION}'):
        if len(d) == 0:
            continue

        c = d[0]

        p = rewriter_ILM_preds[pred_idx]
        _o_idx = c['original_id']
        o = orig_dev_dataset[_o_idx]
        assert ' '.join(c['question_toks']) == p['question'], (' '.join(c['question_toks']), p['question'])
        assert c['gold_question_toks'] == o['question_toks'], (c['gold_question_toks'], o['question_toks'])

        # Debug 
        # assert c['rewriter_tags'] == p['rewriter_tags'][:len(c['rewriter_tags'])], f"{c['rewriter_tags']}\n{p['rewriter_tags']}\nShould raise"

        _db_id = o['db_id']

        # _tags = p['tags_prediction']  # For previous taggerILM joint model 
        # _tags = p['tags']  # Before adding align_tags (when 'tags' refers to 'rewriter_tags')

        _tags = p['rewriter_tags']
        _rewrite_seq = p['rewrite_seq_prediction']
        _question_toks = c['question_toks']

        # _rewritten_question_toks = Postprocess_rewrite_seq(_tags, _rewrite_seq, _question_toks)
        _rewritten_question_toks = ILM_rewrite_func(c, p)
        _rewritten_question = ' '.join(_rewritten_question_toks)

        _pred_sql = Question(_rewritten_question, _db_id, model_dict=model_dicts[rat_sql_run])[0]['inferred_code']

        _gold_sql = c['query']
        _exact, _score, _exec = EvaluateSQL(_pred_sql, _gold_sql, _db_id)

        # Save the taggerILM raw outputs, for later aggregation 
        c['pred_tags'] = p['rewriter_tags']
        c['pred_ILM'] = p['rewrite_seq_prediction']
        c['pred_ILM_cands'] = p['rewrite_seq_prediction_cands']

        # Save prediction results 
        c['rewritten_question'] = p['rewritten_question'] = _rewritten_question
        c['pred_sql'] = p['pred_sql'] = _pred_sql
        p['gold_sql'] = _gold_sql
        c['score'] = p['score'] = _score
        c['exact'] = p['exact'] = _exact
        c['exec'] = p['exec'] = _exec

        _rewritten_question_toks = [_t.lower() for _t in _rewritten_question_toks]
        _gold_question_toks = [_t.lower() for _t in c['gold_question_toks']]

        ref_list.append([_gold_question_toks])
        hyp_list.append(_rewritten_question_toks)

        pred_idx += len(d)

    # Only using the 1st candidate to rewrite 
    _avg_1st = sum([d[0]['score'] for d in test_dataset]) / len(test_dataset)
    _avg_exact_1st = sum([d[0]['exact'] for d in test_dataset]) / len(test_dataset)
    _avg_exec_1st = sum([d[0]['exec'] for d in test_dataset]) / len(test_dataset)

    ## Std-dev (1st cand only)
    _std_1st = np.std([d[0]['score'] for d in test_dataset])

    ## BLEU 
    _bleu = corpus_bleu(list_of_references=ref_list,
                        hypotheses=hyp_list)

    print('='*20, f'VERSION: {VERSION}', '='*20)
    print('avg_exact = {:.4f}'.format(_avg_exact_1st))
    # print('avg = {:.4f} (std = {:.4f})'.format(_avg_1st, _std_1st))
    print('avg = {:.4f}'.format(_avg_1st))
    print('avg_exec = {:.4f}'.format(_avg_exec_1st))
    print(f'BLEU = {_bleu:.4f}')
    print('='*55)
    
    if test_output_path is not None:
        with open(test_output_path, 'w') as f:
            json.dump(test_dataset, f, indent=4)


In [25]:
## Full tagger-ILM

HUMAN_TEST = False
ASR = 'Amazon'

In [264]:
## Old version

# VERSION_LIST = [f'2.12.1.{v}t-2.23.2.{v}i' for v in [0]]
# # VERSION_LIST += [f'2.12.1.{v}t-2.23.1.{v}i' for v in [0,1,2]]
# # VERSION_LIST = [f'2.12.1.{v}t-2.16.{i}.{v}i' for i in [3,4] for v in range(5)]

# for VERSION in VERSION_LIST:
#     if not HUMAN_TEST:
#         if ASR == 'AssemblyAI':
#             rewriter_ILM_pred_path = '/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/AssemblyAI/output-{}.json'.format(VERSION)
#             test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/Assembly_transcribe/test_rewriter.json'
#             orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'
#         else:
#             rewriter_ILM_pred_path = '/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/output-{}.json'.format(VERSION)
#             test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/test_rewriter.json'
#             orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'
#     else:
#         rewriter_ILM_pred_path = '/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/output-humantest-yshao-{}.json'.format(VERSION)
#         test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test_yshao_rewriter.json'
#         orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test.json'

#     with open(rewriter_ILM_pred_path, 'r') as f:
#         rewriter_ILM_preds = [json.loads(l) for l in f.readlines()]
#     with open(test_dataset_path, 'r') as f:
#         test_dataset = json.load(f)
#     with open(orig_dev_path, 'r') as f:
#         orig_dev_dataset = json.load(f)

#     # len(rewriter_ILM_preds), len(test_dataset), sum([len(d) for d in test_dataset]), len(orig_dev_dataset)

#     ## Quick evaluation: only using the 1st ASR candidate

#     pred_idx = 0

#     ref_list = []
#     hyp_list = []

#     for d in tqdm(test_dataset, desc=f'VERSION {VERSION}'):
#         if len(d) == 0:
#             continue

#         c = d[0]

#         p = rewriter_ILM_preds[pred_idx]
#         _o_idx = c['original_id']
#         o = orig_dev_dataset[_o_idx]
#         assert ' '.join(c['question_toks']) == p['question'], (' '.join(c['question_toks']), p['question'])
#         assert c['gold_question_toks'] == o['question_toks'], (c['gold_question_toks'], o['question_toks'])

#         # Debug 
#         # assert c['rewriter_tags'] == p['rewriter_tags'][:len(c['rewriter_tags'])], f"{c['rewriter_tags']}\n{p['rewriter_tags']}\nShould raise"

#         _db_id = o['db_id']

#         # _tags = p['tags_prediction']  # For previous taggerILM joint model 
#         # _tags = p['tags']  # Before adding align_tags (when 'tags' refers to 'rewriter_tags')

#         _tags = p['rewriter_tags']
#         _rewrite_seq = p['rewrite_seq_prediction']
#         _question_toks = c['question_toks']

#         _rewritten_question_toks = Postprocess_rewrite_seq(_tags, _rewrite_seq, _question_toks)
#         _rewritten_question = ' '.join(_rewritten_question_toks)

#         _pred_sql = Question(_rewritten_question, _db_id, model_dict=model_dicts['orig'])[0]['inferred_code']

#         _gold_sql = c['query']
#         _exact, _score, _exec = EvaluateSQL(_pred_sql, _gold_sql, _db_id)

#         # Save the taggerILM raw outputs, for later aggregation 
#         c['pred_tags'] = p['rewriter_tags']
#         c['pred_ILM'] = p['rewrite_seq_prediction']
#         c['pred_ILM_cands'] = p['rewrite_seq_prediction_cands']

#         # Save prediction results 
#         c['rewritten_question'] = p['rewritten_question'] = _rewritten_question
#         c['pred_sql'] = p['pred_sql'] = _pred_sql
#         p['gold_sql'] = _gold_sql
#         c['score'] = p['score'] = _score
#         c['exact'] = p['exact'] = _exact
#         c['exec'] = p['exec'] = _exec

#         _rewritten_question_toks = [_t.lower() for _t in _rewritten_question_toks]
#         _gold_question_toks = [_t.lower() for _t in c['gold_question_toks']]

#         ref_list.append([_gold_question_toks])
#         hyp_list.append(_rewritten_question_toks)

#         pred_idx += len(d)

#     # Only using the 1st candidate to rewrite 
#     _avg_1st = sum([d[0]['score'] for d in test_dataset]) / len(test_dataset)
#     _avg_exact_1st = sum([d[0]['exact'] for d in test_dataset]) / len(test_dataset)
#     _avg_exec_1st = sum([d[0]['exec'] for d in test_dataset]) / len(test_dataset)

#     ## Std-dev (1st cand only)
#     _std_1st = np.std([d[0]['score'] for d in test_dataset])

#     ## BLEU 
#     _bleu = corpus_bleu(list_of_references=ref_list,
#                         hypotheses=hyp_list)

#     print('='*20, f'VERSION: {VERSION}', '='*20)
#     print('avg_exact = {:.4f}'.format(_avg_exact_1st))
#     # print('avg = {:.4f} (std = {:.4f})'.format(_avg_1st, _std_1st))
#     print('avg = {:.4f}'.format(_avg_1st))
#     print('avg_exec = {:.4f}'.format(_avg_exec_1st))
#     print(f'BLEU = {_bleu:.4f}')
#     print('='*55)
    
#     # Non ASR
#     if not HUMAN_TEST:
#         if ASR == 'AssemblyAI':
#             test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/AssemblyAI/ratsql-test-save/{VERSION}.json'
#         else:
#             test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-test-save/{VERSION}.json'
#     else:
#         test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-test-save/humantest-yshao-{VERSION}.json'
#     # ASR
#     # test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-asr-test-save/{VERSION}.json'
#     # Mixed
#     # test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-mixed-test-save/{VERSION}.json'

#     with open(test_output_path, 'w') as f:
#         json.dump(test_dataset, f, indent=4)


In [326]:
VERSION_LIST = [f'2.12.1.{v}t-2.18.3.{v}i' for v in [3,4]]
VERSION_LIST += [f'2.12.1.{v}t-2.19.3.{v}i' for v in [3,4]]
VERSION_LIST += [f'2.12.1.{v}t-2.21.1.{v}i' for v in [2,3,4]]
# VERSION_LIST += [f'2.12.1.{v}t-2.20.0.{v}i' for v in [0,1,2,3,4]]
# VERSION_LIST = [f'2.18.2.{v}i-oracle-tags' for v in range(5)]

for VERSION in VERSION_LIST:
    if not HUMAN_TEST:
        if ASR == 'AssemblyAI':
            rewriter_ILM_pred_path = '/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/AssemblyAI/output-{}.json'.format(VERSION)
            test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/Assembly_transcribe/test_rewriter.json'
            orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'
        else:
            rewriter_ILM_pred_path = '/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/output-{}.json'.format(VERSION)
            test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/test_rewriter.json'
            orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'
    else:
        rewriter_ILM_pred_path = '/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/output-humantest-yshao-{}.json'.format(VERSION)
        test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test_yshao_rewriter.json'
        orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test.json'

    # Non ASR
    if not HUMAN_TEST:
        if ASR == 'AssemblyAI':
            test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/AssemblyAI/ratsql-test-save/{VERSION}.json'
        else:
            test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-test-save/{VERSION}.json'
    else:
        test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-test-save/humantest-yshao-{VERSION}.json'
    # ASR
    # test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-asr-test-save/{VERSION}.json'
    # Mixed
    # test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-mixed-test-save/{VERSION}.json'

    
    Full_evaluate_ILM(eval_version=VERSION,
                      rewriter_ILM_pred_path=rewriter_ILM_pred_path,
                      test_dataset_path=test_dataset_path,
                      orig_dev_path=orig_dev_path,
                      test_output_path=test_output_path)



--- Not enough edits ---
Tags: ['O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'U-EDIT', 'U-EDIT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Edits: [['?']]
--- Not enough edits ---
Tags: ['O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'B-EDIT', 'L-EDIT', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Edits: []
--- Not enough edits ---
Tags: ['O-KEEP', 'O-KEEP', 'O-KEEP', 'U-EDIT',

--- Not enough edits ---
Tags: ['O-KEEP', 'O-KEEP', 'O-KEEP', 'U-EDIT', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'B-EDIT', 'L-EDIT', 'O-KEEP', 'B-EDIT', 'L-EDIT', 'U-EDIT', 'U-DEL', 'U-EDIT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Edits: [['ids'], ["'SF", "'"], ['ids'], ['?']]
--- Not enough edits ---
Tags: ['O-KEEP', 'O-KEEP', 'O-KEEP', 'U-DEL', 'U-EDIT', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'U-EDIT', 'O-KEEP', 'U-EDIT', 'O-KEEP', 'B-DEL', 'L-DEL', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'

--- Not enough edits ---
Tags: ['O-KEEP', 'O-KEEP', 'O-KEEP', 'U-EDIT', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'B-EDIT', 'L-EDIT', 'O-KEEP', 'B-EDIT', 'L-EDIT', 'U-EDIT', 'U-DEL', 'U-EDIT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Edits: [['ids'], ['BK'], ['id'], ['?']]
--- Not enough edits ---
Tags: ['O-KEEP', 'O-KEEP', 'O-KEEP', 'U-DEL', 'U-EDIT', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'U-EDIT', 'O-KEEP', 'B-EDIT', 'I-EDIT', 'L-EDIT', 'O-KEEP', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'

--- Not enough edits ---
Tags: ['U-EDIT', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'B-EDIT', 'L-EDIT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Edits: [['Select']]
--- Not enough edits ---
Tags: ['U-EDIT', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'B-EDIT', 'L-EDIT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Edits: [['Select']]
--- Not enough edits ---
Tags: ['O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'U-DEL', 'B-EDIT', 'L-EDIT', 'B-EDIT', 'L-EDIT', 'U-EDIT', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'U-EDIT', 'B-EDIT', 'L-EDIT', 'U-EDIT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 

--- Not enough edits ---
Tags: ['O-KEEP', 'O-KEEP', 'O-KEEP', 'U-DEL', 'U-EDIT', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'U-EDIT', 'O-KEEP', 'U-EDIT', 'O-KEEP', 'B-DEL', 'L-DEL', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Edits: [['ids'], ['``', 'Canada', "''"]]
--- Not enough edits ---
Tags: ['O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'U-EDIT', 'U-EDIT', 'O-KEEP', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'

--- Not enough edits ---
Tags: ['O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'U-EDIT', 'O-KEEP', 'U-EDIT', 'U-EDIT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Edits: [['a', 'higher'], ['?']]
--- Not enough edits ---
Tags: ['U-EDIT', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'B-EDIT', 'L-EDIT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Edits: [['What', 'are']]
--- Not enough edits ---
Tags: ['O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'B-EDIT', 'L-EDIT', 'U-EDIT', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O

--- Not enough edits ---
Tags: ['O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'U-EDIT', 'O-KEEP', 'U-EDIT', 'B-EDIT', 'L-EDIT', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'U-EDIT', 'O-KEEP', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Edits: [['and'], ['named', "'Kolob"], ["'GT", "'"]]
--- Not enough edits ---
Tags: ['O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'U-EDIT', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'U-EDIT', 'O-KEEP', 'U-EDIT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Edits: [['height'], ['?']]
--- Not enough edits ---
Tags: ['U-EDIT', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O-KEEP', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 

##### Rewrite_seq BLEU/WER

In [104]:
## rewrite_seq BLEU (turns out that allennlp BLEU is different from nltk BLEU)
## also, predicted rewrite_seq is based on predicted tags; thus, use oracle rewrite instead of gold rewrite 

VERSION_LIST = [f'2.12.1.{v}t-2.32.0.{v}i' for v in range(4)]
# VERSION_LIST += [f'2.12.1.{v}t-2.23.1.{v}i' for v in [0,1,2]]
# VERSION_LIST = [f'2.12.1.{v}t-2.16.{i}.{v}i' for i in [3,4] for v in range(5)]

_bleu_list = []
_wer_list = []
_rewrite_bleu_list = []
_rewrite_wer_list = []

for VERSION in VERSION_LIST:
#     if not HUMAN_TEST:
#         if ASR == 'AssemblyAI':
#             rewriter_ILM_pred_path = '/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/AssemblyAI/output-{}.json'.format(VERSION)
#             test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/Assembly_transcribe/test_rewriter.json'
#             orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'
#         else:
#             rewriter_ILM_pred_path = '/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/output-{}.json'.format(VERSION)
#             test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/test_rewriter.json'
#             orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'
#     else:
#         rewriter_ILM_pred_path = '/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/output-humantest-yshao-{}.json'.format(VERSION)
#         test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test_yshao_rewriter.json'
#         orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test.json'

    rewriter_ILM_pred_path = '/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/output-{}.json'.format(VERSION)
    test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/test_rewriter.json'
    orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'

    with open(rewriter_ILM_pred_path, 'r') as f:
        rewriter_ILM_preds = [json.loads(l) for l in f.readlines()]
    with open(test_dataset_path, 'r') as f:
        test_dataset = json.load(f)
    with open(orig_dev_path, 'r') as f:
        orig_dev_dataset = json.load(f)

    # len(rewriter_ILM_preds), len(test_dataset), sum([len(d) for d in test_dataset]), len(orig_dev_dataset)

    ## Quick evaluation: only using the 1st ASR candidate

    pred_idx = 0

    ref_list = []
    hyp_list = []
    wer_numer = 0
    wer_denom = 0
    
    ref_list_rew = []
    hyp_list_rew = []
    wer_numer_rew = 0
    wer_denom_rew = 0

    for d in tqdm(test_dataset, desc=f'VERSION {VERSION}'):
        if len(d) == 0:
            continue

        c = d[0]

        p = rewriter_ILM_preds[pred_idx]
        _o_idx = c['original_id']
        o = orig_dev_dataset[_o_idx]
        assert ' '.join(c['question_toks']).lower() == p['question'].lower(), (' '.join(c['question_toks']), p['question'])
        assert c['gold_question_toks'] == o['question_toks'], (c['gold_question_toks'], o['question_toks'])

        # Debug 
        # assert c['rewriter_tags'] == p['rewriter_tags'][:len(c['rewriter_tags'])], f"{c['rewriter_tags']}\n{p['rewriter_tags']}\nShould raise"

        _db_id = o['db_id']

        # _tags = p['tags_prediction']  # For previous taggerILM joint model 
        # _tags = p['tags']  # Before adding align_tags (when 'tags' refers to 'rewriter_tags')

        _tags = p['rewriter_tags']
        _rewrite_seq = [t for t in p['rewrite_seq_prediction'] if t != END_SYMBOL] + [END_SYMBOL]
        
#         _gold_rewrite_list = []
#         for _edit in c['rewriter_edits']:
#             _gold_rewrite_list.extend(_edit['tgt_text'].split(' '))
#             _gold_rewrite_list.append('[ANS]')
#         _gold_rewrite_list.append(END_SYMBOL)

#         _save = dict()
#         Oracle_ILM_rewrite(c, p, rewrite_seq_toks_save=_save)
#         _gold_rewrite_list = _save['rewrite_seq_toks']
#         _gold_rewrite_list.append(END_SYMBOL)
        
#         _gold_rewrite_list = [_t.lower() for _t in _gold_rewrite_list]
        _rewrite_seq = [_t.lower() for _t in _rewrite_seq]
        
        ref_list_rew.append([_gold_rewrite_list])
        hyp_list_rew.append(_rewrite_seq)
        wer_numer_rew += editdistance.eval(_gold_rewrite_list, _rewrite_seq)
        wer_denom_rew += len(_gold_rewrite_list)
        
#         print(p['question'])
#         print(_rewrite_seq)
#         print(_gold_rewrite_list)

#         _rewritten_question_toks = _Postprocess_rewrite_seq_wrapper(c, p)
        _rewritten_question_toks = p['rewrite_seq_prediction']
        _rewritten_question_toks = [_t.lower() for _t in _rewritten_question_toks]
        _gold_question_toks = [_t.lower() for _t in c['gold_question_toks']]
        
        ref_list.append([_gold_question_toks])
        hyp_list.append(_rewritten_question_toks)
        wer_numer += editdistance.eval(_gold_question_toks, _rewritten_question_toks)
        wer_denom += len(_gold_question_toks)
        
        pred_idx += len(d)
        
    ## BLEU 
#     _rewrite_bleu = corpus_bleu(list_of_references=ref_list_rew,
#                         hypotheses=hyp_list_rew)
#     _rewrite_bleu_list.append(_rewrite_bleu)
    
    _bleu = corpus_bleu(list_of_references=ref_list,
                        hypotheses=hyp_list)
    _bleu_list.append(_bleu)
    
    _wer = wer_numer / wer_denom
    _wer_list.append(_wer)
    
#     _wer_rew = wer_numer_rew / wer_denom_rew
#     _rewrite_wer_list.append(_wer_rew)
    
    print('='*20, f'VERSION: {VERSION}', '='*20)
    print(f'BLEU = {_bleu:.4f}')
    print(f'WER = {wer_numer} / {wer_denom} = {_wer:.4f}')
#     print(f'Rewrite_seq BLEU = {_rewrite_bleu:.4f}')
#     print(f'Rewrite_seq WER = {wer_numer_rew} / {wer_denom_rew} = {_wer_rew:.4f}')
    print('='*55)




==================== VERSION: 2.12.1.0t-2.32.0.0i ====================
BLEU = 0.8740
WER = 532 / 7146 = 0.0744



==================== VERSION: 2.12.1.1t-2.32.0.1i ====================
BLEU = 0.8857
WER = 469 / 7146 = 0.0656



==================== VERSION: 2.12.1.2t-2.32.0.2i ====================
BLEU = 0.8734
WER = 514 / 7146 = 0.0719



==================== VERSION: 2.12.1.3t-2.32.0.3i ====================
BLEU = 0.8822
WER = 479 / 7146 = 0.0670


In [106]:
# print('\n'.join([f'{p[0]:.4f}\t{p[1]:.4f}\t{p[2]:.4f}\t{p[3]:.4f}' for p in zip(_bleu_list, _wer_list, _rewrite_bleu_list, _rewrite_wer_list)]))
print('\n'.join([f'{p[0]:.4f}\t{p[1]:.4f}' for p in zip(_bleu_list, _wer_list)]))


0.8740	0.0744
0.8857	0.0656
0.8734	0.0719
0.8822	0.0670


In [66]:
corpus_bleu([[[1,2,3]]], [[1,2,3]], smoothing_function=None)

/Users/mac/.pyenv/versions/3.7.5/envs/py3.7-speakql/lib/python3.7/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


1.2213386697554703e-77

##### Freeze/Modify POS
- TODO: use func

In [322]:
class _Postprocess_POS_wrapper:
    def __init__(self, POS, mode):
        self.POS = POS
        assert mode in ('freeze', 'modify'), mode
        self.mode = mode
    
    def __call__(self, cand_dict, pred_dict):
        _tags = pred_dict['rewriter_tags']
        _rewrite_seq = pred_dict['rewrite_seq_prediction']
        _question_toks = cand_dict['question_toks']
        
        if self.mode == 'freeze':
            _rewritten_toks = Postprocess_rewrite_seq_freeze_POS(
                _tags,
                _rewrite_seq,
                _question_toks,
                freeze_POS=self.POS,
                nlp=nlp)
        else:
            _rewritten_toks = Postprocess_rewrite_seq_modify_POS(
                _tags,
                _rewrite_seq,
                _question_toks,
                modify_POS=self.POS,
                nlp=nlp)
        return _rewritten_toks

In [323]:
VERSION_LIST = [f'2.12.1.{v}t-2.18.2.{v}i' for v in [2]]

POS_LIST = ["PUNCT", "NUM", "VERB", "PRON", "ADP", "NOUN", "AUX", "DET",
            "SCONJ", "PART", "ADJ", "CCONJ", "ADV", "PROPN"]

GROUP_LIST = [(_v, _p) for _v in VERSION_LIST for _p in POS_LIST]

for VERSION, POS in GROUP_LIST:
    rewriter_ILM_pred_path = '/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/output-{}.json'.format(VERSION)
    test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/test_rewriter.json'
    orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'
    test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-test-save/{VERSION}-freeze={POS}.json'
    
    _rewriter = _Postprocess_POS_wrapper(POS, 'freeze')
    
    Full_evaluate_ILM(eval_version=f"Freeze-{POS}-{VERSION}",
                      rewriter_ILM_pred_path=rewriter_ILM_pred_path,
                      test_dataset_path=test_dataset_path,
                      orig_dev_path=orig_dev_path,
                      test_output_path=test_output_path,
                      ILM_rewrite_func=_rewriter)

    with open(test_output_path, 'w') as f:
        json.dump(test_dataset, f, indent=4)



SELECT Max(performance.Share), Min(performance.Official_ratings_(millions)) FROM performance WHERE performance.Type != 'terminal'
SELECT max(SHARE) ,  min(SHARE) FROM performance WHERE TYPE != "Live final"
orchestra
process_sql.get_sql() failed

==================== VERSION: Freeze-PUNCT-2.12.1.2t-2.18.2.2i ====================
avg_exact = 0.5082
avg = 0.7587
avg_exec = 0.3766
BLEU = 0.8451


SELECT Max(performance.Share), Min(performance.Official_ratings_(millions)) FROM performance WHERE performance.Type != 'terminal'
SELECT max(SHARE) ,  min(SHARE) FROM performance WHERE TYPE != "Live final"
orchestra
process_sql.get_sql() failed

==================== VERSION: Freeze-NUM-2.12.1.2t-2.18.2.2i ====================
avg_exact = 0.5137
avg = 0.7629
avg_exec = 0.3766
BLEU = 0.8680


SELECT Max(performance.Share), Min(performance.Official_ratings_(millions)) FROM performance WHERE performance.Type != 'terminal'
SELECT max(SHARE) ,  min(SHARE) FROM performance WHERE TYPE != "Live final"
orchestra
process_sql.get_sql() failed

==================== VERSION: Freeze-VERB-2.12.1.2t-2.18.2.2i ====================
avg_exact = 0.5247
avg = 0.7712
avg_exec = 0.3748
BLEU = 0.8679


SELECT Max(performance.Share), Min(performance.Official_ratings_(millions)) FROM performance WHERE performance.Type != 'terminal'
SELECT max(SHARE) ,  min(SHARE) FROM performance WHERE TYPE != "Live final"
orchestra
process_sql.get_sql() failed

==================== VERSION: Freeze-PRON-2.12.1.2t-2.18.2.2i ====================
avg_exact = 0.5155
avg = 0.7654
avg_exec = 0.3711
BLEU = 0.8654


SELECT Max(performance.Share), Min(performance.Official_ratings_(millions)) FROM performance WHERE performance.Type != 'terminal'
SELECT max(SHARE) ,  min(SHARE) FROM performance WHERE TYPE != "Live final"
orchestra
process_sql.get_sql() failed

==================== VERSION: Freeze-ADP-2.12.1.2t-2.18.2.2i ====================
avg_exact = 0.5174
avg = 0.7621
avg_exec = 0.3729
BLEU = 0.8657


...

SELECT Max(performance.Share), Min(performance.Official_ratings_(millions)) FROM performance WHERE performance.Type != 'terminal'
SELECT max(SHARE) ,  min(SHARE) FROM performance WHERE TYPE != "Live final"
orchestra
process_sql.get_sql() failed

==================== VERSION: Freeze-PART-2.12.1.2t-2.18.2.2i ====================
avg_exact = 0.5265
avg = 0.7714
avg_exec = 0.3784
BLEU = 0.8729


SELECT Max(performance.Share), Min(performance.Official_ratings_(millions)) FROM performance WHERE performance.Type != 'terminal'
SELECT max(SHARE) ,  min(SHARE) FROM performance WHERE TYPE != "Live final"
orchestra
process_sql.get_sql() failed

==================== VERSION: Freeze-ADJ-2.12.1.2t-2.18.2.2i ====================
avg_exact = 0.5265
avg = 0.7696
avg_exec = 0.3784
BLEU = 0.8725


SELECT Max(performance.Share), Min(performance.Official_ratings_(millions)) FROM performance WHERE performance.Type != 'terminal'
SELECT max(SHARE) ,  min(SHARE) FROM performance WHERE TYPE != "Live final"
orchestra
process_sql.get_sql() failed

==================== VERSION: Freeze-CCONJ-2.12.1.2t-2.18.2.2i ====================
avg_exact = 0.5265
avg = 0.7711
avg_exec = 0.3784
BLEU = 0.8695



==================== VERSION: Freeze-ADV-2.12.1.2t-2.18.2.2i ====================
avg_exact = 0.5302
avg = 0.7744
avg_exec = 0.3821
BLEU = 0.8730


SELECT Max(performance.Share), Min(performance.Official_ratings_(millions)) FROM performance WHERE performance.Type != 'terminal'
SELECT max(SHARE) ,  min(SHARE) FROM performance WHERE TYPE != "Live final"
orchestra
process_sql.get_sql() failed

==================== VERSION: Freeze-PROPN-2.12.1.2t-2.18.2.2i ====================
avg_exact = 0.5174
avg = 0.7664
avg_exec = 0.3766
BLEU = 0.8740


In [324]:
VERSION_LIST = [f'2.12.1.{v}t-2.18.2.{v}i' for v in [2]]

POS_LIST = ["PUNCT", "NUM", "VERB", "PRON", "ADP", "NOUN", "AUX", "DET",
            "SCONJ", "PART", "ADJ", "CCONJ", "ADV", "PROPN"]

GROUP_LIST = [(_v, _p) for _v in VERSION_LIST for _p in POS_LIST]

for VERSION, POS in GROUP_LIST:
    rewriter_ILM_pred_path = '/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/output-{}.json'.format(VERSION)
    test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/test_rewriter.json'
    orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'
    test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-test-save/{VERSION}-modify={POS}.json'
    
    _rewriter = _Postprocess_POS_wrapper(POS, 'modify')
    
    Full_evaluate_ILM(eval_version=f"Modify-{POS}-{VERSION}",
                      rewriter_ILM_pred_path=rewriter_ILM_pred_path,
                      test_dataset_path=test_dataset_path,
                      orig_dev_path=orig_dev_path,
                      test_output_path=test_output_path,
                      ILM_rewrite_func=_rewriter)

    with open(test_output_path, 'w') as f:
        json.dump(test_dataset, f, indent=4)


==================== VERSION: Modify-PUNCT-2.12.1.2t-2.18.2.2i ====================
avg_exact = 0.4753
avg = 0.7387
avg_exec = 0.3419
BLEU = 0.8257



==================== VERSION: Modify-NUM-2.12.1.2t-2.18.2.2i ====================
avg_exact = 0.4698
avg = 0.7353
avg_exec = 0.3419
BLEU = 0.8055



==================== VERSION: Modify-VERB-2.12.1.2t-2.18.2.2i ====================
avg_exact = 0.4607
avg = 0.7292
avg_exec = 0.3437
BLEU = 0.8057



==================== VERSION: Modify-PRON-2.12.1.2t-2.18.2.2i ====================
avg_exact = 0.4698
avg = 0.7347
avg_exec = 0.3492
BLEU = 0.8058



==================== VERSION: Modify-ADP-2.12.1.2t-2.18.2.2i ====================
avg_exact = 0.4662
avg = 0.7365
avg_exec = 0.3455
BLEU = 0.8077


...


==================== VERSION: Modify-PART-2.12.1.2t-2.18.2.2i ====================
avg_exact = 0.4570
avg = 0.7273
avg_exec = 0.3400
BLEU = 0.8010



==================== VERSION: Modify-ADJ-2.12.1.2t-2.18.2.2i ====================
avg_exact = 0.4570
avg = 0.7269
avg_exec = 0.3400
BLEU = 0.8013



==================== VERSION: Modify-CCONJ-2.12.1.2t-2.18.2.2i ====================
avg_exact = 0.4570
avg = 0.7273
avg_exec = 0.3400
BLEU = 0.8042


SELECT Max(performance.Share), Min(performance.Official_ratings_(millions)) FROM performance WHERE performance.Type != 'terminal'
SELECT max(SHARE) ,  min(SHARE) FROM performance WHERE TYPE != "Live final"
orchestra
process_sql.get_sql() failed

==================== VERSION: Modify-ADV-2.12.1.2t-2.18.2.2i ====================
avg_exact = 0.4534
avg = 0.7244
avg_exec = 0.3364
BLEU = 0.8009



==================== VERSION: Modify-PROPN-2.12.1.2t-2.18.2.2i ====================
avg_exact = 0.4735
avg = 0.7318
avg_exec = 0.3400
BLEU = 0.8004


##### Trained tagger + Oracle ILM

In [32]:
def Oracle_ILM_rewrite(cand_dict, pred_dict, rewrite_seq_toks_save=None):
    ## rewrite_seq_toks_save: Dict, used to save the computed rewrite_seq_toks 
    
    tags = pred_dict['rewriter_tags']
    question_toks = cand_dict['question_toks']
    
    gold_toks = cand_dict['gold_question_toks']
    alignment_span_pairs = cand_dict['alignment_span_pairs']  # List[Tuple[List: src_span, List: tgt_span]]
    question_toks_rewritten = []
    rewrite_seq_toks = []

    _last_src_idx = -1
    for src_span, tgt_span in alignment_span_pairs:
        for i in range(_last_src_idx + 1, src_span[0]):
            # ignored src tokens 
            if tags[i].endswith('KEEP'):
                question_toks_rewritten.append(question_toks[i])
            else:
                # del or edit 
                pass
        
        if len(src_span) == len(tgt_span) or len(src_span) == 1:
            # treat src as single tokens 
            if len(src_span) == len(tgt_span):
                _src_spans = [[_idx] for _idx in src_span]
                _tgt_spans = [[_idx] for _idx in tgt_span]
            else:
                _src_spans = [src_span]
                _tgt_spans = [tgt_span]
                
            for _src_span, _tgt_span in zip(_src_spans, _tgt_spans):
                _idx = _src_span[0]
                if tags[_idx].endswith('KEEP'):
                    question_toks_rewritten.append(question_toks[_idx])
                elif tags[_idx].endswith('DEL'):
                    pass
                elif tags[_idx].endswith('EDIT'):
                    # edit to correct, i.e. append corresponding gold tokens 
                    for j in _tgt_span:
                        question_toks_rewritten.append(gold_toks[j])
                        rewrite_seq_toks.append(gold_toks[j])
                        if tags[_idx] in {'U-EDIT', 'L-EDIT'}:
                            rewrite_seq_toks.append('[ANS]')
        else:
            # multi-token            
            if all([tags[i].endswith('DEL') for i in src_span]):
                # if all del, del 
                pass
            elif all([tags[i].endswith('DEL') or tags[i].endswith('EDIT') for i in src_span]):
                # if all edit or del (at least 1 edit), edit 
                for j in tgt_span:
                    question_toks_rewritten.append(gold_toks[j])
            else:
                # otherwise (if has keep), keep; if not all keep, print warning 
                assert any([tags[i].endswith('KEEP') for i in src_span])
                if not all([tags[i].endswith('KEEP') for i in src_span]):
                    print(f'Unaddressed mismatch (treated as KEEP): {pred_dict["original_id"]}')
                    print('Target', [gold_toks[j] for j in tgt_span])
                    print('Source', [(question_toks[i], tags[i]) for i in src_span])

        _last_src_idx = src_span[-1]
                    
    if rewrite_seq_toks_save is not None:
        rewrite_seq_toks_save['rewrite_seq_toks'] = rewrite_seq_toks
            
    return question_toks_rewritten



In [286]:
for s in test_dataset:
    c = s[0]
    if c['original_id'] == 193:
        break

In [287]:
for p in rewriter_ILM_preds:
    if p['original_id'] == 193:
        break

In [288]:
Oracle_ILM_rewrite(c, p)

Unaddressed mismatch: 193
['abbreviation', "'UAL", "'"]
[('abbreviation', 'O-KEEP'), ("you'il", 'U-EDIT')]


['which', 'airline', 'has', '?']

In [ ]:
c

In [ ]:
p

In [ ]:
VERSION_LIST = [f'2.12.1.{v}t-2.18.2.{v}i' for v in [0,1,2,3,4]]

for VERSION in VERSION_LIST:
    if not HUMAN_TEST:
        if ASR == 'AssemblyAI':
            rewriter_ILM_pred_path = '/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/AssemblyAI/output-{}.json'.format(VERSION)
            test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/Assembly_transcribe/test_rewriter.json'
            orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'
        else:
            rewriter_ILM_pred_path = '/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/output-{}.json'.format(VERSION)
            test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/test_rewriter.json'
            orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'
    else:
        rewriter_ILM_pred_path = '/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/output-humantest-yshao-{}.json'.format(VERSION)
        test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test_yshao_rewriter.json'
        orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test.json'

    _tagger_ver = VERSION.split('-')[0]
    
    # Non ASR
    if not HUMAN_TEST:
        if ASR == 'AssemblyAI':
            test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/AssemblyAI/ratsql-test-save/{_tagger_ver}-oracle-ILM.json'
        else:
            test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-test-save/{_tagger_ver}-oracle-ILM.json'
    else:
        test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-test-save/humantest-yshao-{_tagger_ver}-oracle-ILM.json'
    # ASR
    # test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-asr-test-save/{_tagger_ver}-oracle-ILM.json'
    # Mixed
    # test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-mixed-test-save/{_tagger_ver}-oracle-ILM.json'

        
    Full_evaluate_ILM(eval_version=VERSION,
                      rewriter_ILM_pred_path=rewriter_ILM_pred_path,
                      test_dataset_path=test_dataset_path,
                      orig_dev_path=orig_dev_path,
                      test_output_path=test_output_path,
                      ILM_rewrite_func=Oracle_ILM_rewrite)




##### Output to test_rewriter (no eval)
- Use py script, based on ILM-output-to-dataset.py

#### Loading from predicted file (only 1st cand is predicted!)

In [ ]:
VERSION = '3.6.0.2'

test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-test-save/{VERSION}.json'

with open(test_output_path, 'r') as f:
    test_dataset = json.load(f)

len(test_dataset), test_dataset[0][0].keys()

In [ ]:
# Using EvaluateSQL_full 

# tables_json = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/tables.json'
# kmaps = evaluation.build_foreign_key_map_from_json(tables_json)

plist = [d[0]['pred_sql'] for d in test_dataset]
glist = [d[0]['query'] for d in test_dataset]
db_id_list = [d[0]['db_id'] for d in test_dataset]

EvaluateSQL_full(glist=glist,
                 plist=plist,
                 db_id_list=db_id_list,
                 kmaps=kmaps)


In [ ]:
# For reranker output file 
VERSION = '1.10.0.2'

test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-test-save/{VERSION}.json'

with open(test_output_path, 'r') as f:
    test_dataset = json.load(f)

len(test_dataset), test_dataset[0][0].keys()

In [ ]:
plist = [c['ratsql_pred_sql'] for d in test_dataset for c in d if c['is_reranker_selection'] == 1]
glist = [d[0]['query'] for d in test_dataset]
db_id_list = [d[0]['db_id'] for d in test_dataset]
assert len(plist) == len(glist) == len(db_id_list)

EvaluateSQL_full(glist=glist,
                 plist=plist,
                 db_id_list=db_id_list,
                 kmaps=kmaps)

In [ ]:
plist = [d[0]['ratsql_pred_sql'] for d in test_dataset]
glist = [d[0]['query'] for d in test_dataset]
db_id_list = [d[0]['db_id'] for d in test_dataset]
assert len(plist) == len(glist) == len(db_id_list)

EvaluateSQL_full(glist=glist,
                 plist=plist,
                 db_id_list=db_id_list,
                 kmaps=kmaps)

In [ ]:
# Using all ASR candidates (no longer in use)

pred_idx = 0

for d in tqdm(test_dataset):
    for c in d:
        p = rewriter_ILM_preds[pred_idx]
        _o_idx = c['original_id']
        o = orig_dev_dataset[_o_idx]
        assert ' '.join(c['question_toks']) == p['question'], (' '.join(c['question_toks']), p['question'])
        assert c['gold_question_toks'] == o['question_toks'], (c['gold_question_toks'], o['question_toks'])
        
        _db_id = o['db_id']

        # _tags = p['tags_prediction']
        _tags = p['tags']
        _rewrite_seq = p['rewrite_seq_prediction']
        _question_toks = c['question_toks']
        
        _rewritten_question_toks = Postprocess_rewrite_seq(_tags, _rewrite_seq, _question_toks)
        _rewritten_question = ' '.join(_rewritten_question_toks)
        
        _pred_sql = Question(_rewritten_question, _db_id)[0]['inferred_code']
        
        _gold_sql = c['query']
        _score = EvaluateSQL(_pred_sql, _gold_sql, _db_id)
        
        c['rewritten_question'] = p['rewritten_question'] = _rewritten_question
        c['pred_sql'] = p['pred_sql'] = _pred_sql
        p['gold_sql'] = _gold_sql
        c['score'] = p['score'] = _score

        pred_idx += 1



In [ ]:
# Using all the candidates to rewrite 
print(sum([p['score'] for p in rewriter_ILM_preds]) / len(rewriter_ILM_preds))
print(sum([c['score'] for d in test_dataset for c in d]) / sum([len(d) for d in test_dataset]))

# Only using the 1st candidate to rewrite 
_avg_1st = sum([d[0]['score'] for d in test_dataset]) / len(test_dataset)

## Std-dev (1st cand only)
_std_1st = np.std([d[0]['score'] for d in test_dataset])

print('avg = {:.4f} (std = {:.4f})'.format(_avg_1st, _std_1st))

In [ ]:
# Evaluation process with oracle tags (no longer in use for version>=2.3.0)

pred_idx = 0

for d in tqdm(test_dataset):
    for c in d:
        p = rewriter_ILM_preds[pred_idx]
        _o_idx = c['original_id']
        o = orig_dev_dataset[_o_idx]
        assert ' '.join(c['question_toks']) == p['question'], (' '.join(c['question_toks']), p['question'])
        assert c['gold_question_toks'] == o['question_toks'], (c['gold_question_toks'], o['question_toks'])
        
        _db_id = o['db_id']

        _tags = p['gold_tags']
        _rewrite_seq = p['oracle_tags_rewrite_seq_prediction']
        _question_toks = c['question_toks']
        
        _rewritten_question_toks = Postprocess_rewrite_seq(_tags, _rewrite_seq, _question_toks)
        _rewritten_question = ' '.join(_rewritten_question_toks)
        
        _pred_sql = Question(_rewritten_question, _db_id)[0]['inferred_code']
        
        _gold_sql = c['query']
        _score = EvaluateSQL(_pred_sql, _gold_sql, _db_id)
        
        c['oracle_tags_rewritten_question'] = p['oracle_tags_rewritten_question'] = _rewritten_question
        c['oracle_tags_pred_sql'] = p['oracle_tags_pred_sql'] = _pred_sql
        c['oracle_tags_score'] = p['oracle_tags_score'] = _score

        pred_idx += 1



In [ ]:
# Using all the candidates to rewrite 
print(sum([p['oracle_tags_score'] for p in rewriter_ILM_preds]) / len(rewriter_ILM_preds))
print(sum([c['oracle_tags_score'] for d in test_dataset for c in d]) / sum([len(d) for d in test_dataset]))

# Only using the 1st candidate to rewrite 
_oracle_avg_1st = sum([d[0]['oracle_tags_score'] for d in test_dataset]) / len(test_dataset)

## Std-dev (1st cand only)
_oracle_std_1st = np.std([d[0]['oracle_tags_score'] for d in test_dataset])

print('avg = {:.4f} (std = {:.4f})'.format(_oracle_avg_1st, _oracle_std_1st))

In [ ]:
# Merge results in a single dataset obj 

test_pred_dataset = []

pred_idx = 0

for d in tqdm(test_dataset):
    _pred_d = []
    
    for c in d:
        p = rewriter_ILM_preds[pred_idx]
        _o_idx = c['original_id']
        o = orig_dev_dataset[_o_idx]
        assert ' '.join(c['question_toks']) == p['question'], (' '.join(c['question_toks']), p['question'])
        assert c['gold_question_toks'] == o['question_toks'], (c['gold_question_toks'], o['question_toks'])
        
        _pred_c = dict()
        
        _pred_c['ASR_question'] = p['question']
        _pred_c['ASR_question_pred_sql'] = c['ratsql_pred_sql']
        
        _pred_c['gold_question'] = c['gold_question']
        # _pred_c['gold_question_pred_sql'] = orig_dev_preds[c['original_id']]
        
        _pred_c['tag_prediction'] = list(zip(p['question'].split(' '), p['tags_prediction']))
        _pred_c['rewrite_seq'] = []
        for t in p['rewrite_seq_prediction']:
            _pred_c['rewrite_seq'].append(t)
            if t == '@end@': break
        _pred_c['rewritten_question'] = p['rewritten_question']
        _pred_c['pred_sql'] = p['pred_sql']
        _pred_c['score'] = p['score']
        
        _pred_c['gold_tags'] = list(zip(p['question'].split(' '), p['gold_tags']))
        _pred_c['oracle_tags_rewrite_seq'] = []
        for t in p['oracle_tags_rewrite_seq_prediction']:
            _pred_c['oracle_tags_rewrite_seq'].append(t)
            if t == '@end@': break
        _pred_c['oracle_tags_rewritten_question'] = p['oracle_tags_rewritten_question']
        _pred_c['oracle_tags_pred_sql'] = p['oracle_tags_pred_sql']
        _pred_c['oracle_tags_score'] = p['oracle_tags_score']
        
        _pred_c['gold_sql'] = c['query']
        
        _pred_d.append(_pred_c)

        pred_idx += 1
    
    test_pred_dataset.append(_pred_d)

len(test_pred_dataset), sum([len(d) for d in test_pred_dataset])

In [ ]:
with open('./output/test-prediction-{}.json'.format(VERSION), 'w') as f:
    json.dump(test_pred_dataset, f, indent=4)

In [ ]:
# Load dataset file with predictions 

with open('./output/pred-{}.json'.format(VERSION), 'r') as f:
    test_pred_dataset = json.load(f)
len(test_pred_dataset), sum([len(d) for d in test_pred_dataset])

In [ ]:
test_pred_dataset[0][0].keys()

In [ ]:
# Analysis 
orig_dev_preds_path = './output/dev_output.txt'

with open(orig_dev_preds_path, 'r') as f:
    orig_dev_preds = [l.strip() for l in f.readlines()]

len(orig_dev_preds)

In [ ]:
# for d in test_dataset[5::40]:
#     print('DB:', d[0]['db_id'])
#     print('ASR question:\t\t', d[0]['question'])
#     print('Rewritten question:\t', d[0]['rewritten_question'])
#     print('Gold question:\t\t', d[0]['gold_question'])
#     print('ASR-q Pred SQL:\t\t', d[0]['ratsql_pred_sql'])
#     print('Rewritten-q Pred SQL:\t', d[0]['pred_sql'])
#     print('Gold-q Pred SQL:\t', orig_dev_preds[d[0]['original_id']])
#     print('Gold SQL:\t\t', d[0]['query'])
#     print('Score:', d[0]['score'])
#     print('-'*50)

In [ ]:
test_pred_samples = [c for d in test_pred_dataset for c in d]
for i, c in list(enumerate(test_pred_samples))[8::88]:
    print('-'*30, 'ID = {}'.format(i), '-'*30)
    print('ASR question:\t\t', c['ASR_question'])
    print('Rewritten question:\t', c['rewritten_question'])
    print('Gold question:\t\t', c['gold_question'])
    print('Rewritten-q Pred SQL:\t', c['pred_sql'])
    print('Gold-q Pred SQL:\t', c['gold_question_pred_sql'])
    print('Gold SQL:\t\t', c['gold_sql'])
    print('Score:', c['score'])
    

In [ ]:
inspect_ids = [8, 96, 272, 448, 1416, 1592, 1680, 1856, 2120, 2296, 2384, 2560, 2824]

#### Seq2seq

In [34]:
## Batch evaluating 

VERSION_LIST = ['3.12.1.1', '3.12.1.2']
HUMAN_TEST = False
ASR = 'Amazon'

for VERSION in VERSION_LIST:
    print(f'Evaluating ({ASR}){" (human test)" if HUMAN_TEST else ""}: VERSION = {VERSION}')
    
    if not HUMAN_TEST:
        if ASR == 'AssemblyAI':
            rewriter_s2s_pred_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/AssemblyAI/output-{VERSION}.json'
            test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/Assembly_transcribe/test_rewriter.json'
            orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'
        else:
            rewriter_s2s_pred_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/output-{VERSION}.json'
            test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/test_rewriter.json'
            orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'
    else:
        # human test 
        rewriter_s2s_pred_path = '/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/output-humantest-yshao-{}.json'.format(VERSION)
        test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test_yshao_rewriter.json'
        orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/human_test/human_test.json'

    
    with open(rewriter_s2s_pred_path, 'r') as f:
        rewriter_preds = [json.loads(l) for l in f.readlines()]
    with open(test_dataset_path, 'r') as f:
        test_dataset = json.load(f)
    with open(orig_dev_path, 'r') as f:
        orig_dev_dataset = json.load(f)
        
    # Quick evaluation: only using the 1st ASR candidate

    ref_list = []
    hyp_list = []
    
    pred_idx = 0

    for d in tqdm(test_dataset):
        if len(d) == 0:
            continue

        c = d[0]

        p = rewriter_preds[pred_idx]
        _o_idx = c['original_id']
        o = orig_dev_dataset[_o_idx]
        assert ' '.join(c['question_toks']) == p['question'], (' '.join(c['question_toks']), p['question'])
        assert c['gold_question_toks'] == o['question_toks'], (c['gold_question_toks'], o['question_toks'])

        _db_id = o['db_id']

        # _tags = p['tags']
        # _rewrite_seq = p['rewrite_seq_prediction']
        # _question_toks = c['question_toks']

        # _rewritten_question_toks = Postprocess_rewrite_seq(_tags, _rewrite_seq, _question_toks)
        # _rewritten_question = ' '.join(_rewritten_question_toks)

        _rewritten_question = ' '.join(p['s2s_prediction'])

        if _rewritten_question == '':
            print(f'_rewritten_question is empty')
            _pred_sql = ''
            _gold_sql = c['query']
            _exact = _score = _exec = 0
        else:
            _pred_sql = Question(_rewritten_question, _db_id)[0]['inferred_code']
            _gold_sql = c['query']
            _exact, _score, _exec = EvaluateSQL(_pred_sql, _gold_sql, _db_id)

        c['rewritten_question'] = p['rewritten_question'] = _rewritten_question
        c['pred_sql'] = p['pred_sql'] = _pred_sql
        p['gold_sql'] = _gold_sql
        c['score'] = p['score'] = _score
        c['exact'] = p['exact'] = _exact
        c['exec'] = p['exec'] = _exec
        
        # For BLEU 
        _rewritten_question_toks = [_t.lower() for _t in p['s2s_prediction']]
        _question_toks = [_t.lower() for _t in c['question_toks']]
        _gold_question_toks = [_t.lower() for _t in c['gold_question_toks']]

        ref_list.append([_gold_question_toks])
        hyp_list.append(_rewritten_question_toks)

        pred_idx += len(d)

    # Only using the 1st candidate to rewrite 
    _avg_1st = sum([d[0]['score'] for d in test_dataset]) / len(test_dataset)
    _avg_exact_1st = sum([d[0]['exact'] for d in test_dataset]) / len(test_dataset)
    _avg_exec_1st = sum([d[0]['exec'] for d in test_dataset]) / len(test_dataset)

    ## Std-dev (1st cand only)
    _std_1st = np.std([d[0]['score'] for d in test_dataset])
    
    ## BLEU 
    _bleu = corpus_bleu(list_of_references=ref_list,
                        hypotheses=hyp_list)
    
    print(f'VERSION {VERSION}:')
    print(f'avg = {_avg_1st:.4f} (std = {_std_1st:.4f})')
    print(f'avg_exact = {_avg_exact_1st:.4f}')
    print(f'avg_exec = {_avg_exec_1st:.4f}')
    print(f'BLEU = {_bleu:.4f}')
    print()
    
    if not HUMAN_TEST:
        if ASR == 'AssemblyAI':
            test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/AssemblyAI/ratsql-test-save/{VERSION}.json'
        else:
            test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-test-save/{VERSION}.json'
    else:
        test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-test-save/humantest-yshao-{VERSION}.json'
        
    with open(test_output_path, 'w') as f:
        json.dump(test_dataset, f, indent=4)


Evaluating (Amazon): VERSION = 3.12.1.1



VERSION 3.12.1.1:
avg = 0.7410 (std = 0.3078)
avg_exact = 0.4826
avg_exec = 0.3638
BLEU = 0.8532

Evaluating (Amazon): VERSION = 3.12.1.2


SELECT Max(performance.Share), Min(performance.Official_ratings_(millions)) FROM performance WHERE performance.Type != 'terminal'
SELECT max(SHARE) ,  min(SHARE) FROM performance WHERE TYPE != "Live final"
orchestra
process_sql.get_sql() failed

VERSION 3.12.1.2:
avg = 0.7478 (std = 0.3037)
avg_exact = 0.4899
avg_exec = 0.3675
BLEU = 0.8566



In [ ]:
# Single version, not used 

VERSION = '3.3.0.0'

rewriter_s2s_pred_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/output-{VERSION}.json'
test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/test_rewriter.json'
orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'


In [ ]:
with open(rewriter_s2s_pred_path, 'r') as f:
    rewriter_preds = [json.loads(l) for l in f.readlines()]
with open(test_dataset_path, 'r') as f:
    test_dataset = json.load(f)
with open(orig_dev_path, 'r') as f:
    orig_dev_dataset = json.load(f)

len(rewriter_preds), len(test_dataset), sum([len(d) for d in test_dataset]), len(orig_dev_dataset)

In [ ]:
rewriter_preds[0].keys()

In [ ]:
for p in rewriter_preds[3::300]:
    print(p['question'])
    print(' '.join(p['s2s_prediction']))
    print(' '.join(p['gold_rewrite_seq_s2s'][1:-1]))
    print('-'*50)

In [ ]:
# Quick evaluation: only using the 1st ASR candidate

pred_idx = 0

for d in tqdm(test_dataset):
    if len(d) == 0:
        continue
        
    c = d[0]
    
    p = rewriter_preds[pred_idx]
    _o_idx = c['original_id']
    o = orig_dev_dataset[_o_idx]
    assert ' '.join(c['question_toks']) == p['question'], (' '.join(c['question_toks']), p['question'])
    assert c['gold_question_toks'] == o['question_toks'], (c['gold_question_toks'], o['question_toks'])

    _db_id = o['db_id']

    # _tags = p['tags']
    # _rewrite_seq = p['rewrite_seq_prediction']
    # _question_toks = c['question_toks']

    # _rewritten_question_toks = Postprocess_rewrite_seq(_tags, _rewrite_seq, _question_toks)
    # _rewritten_question = ' '.join(_rewritten_question_toks)
    
    _rewritten_question = ' '.join(p['s2s_prediction'])
    
    if _rewritten_question == '':
        print(f'_rewritten_question is empty')
        _pred_sql = ''
        _gold_sql = c['query']
        _score = 0
    else:
        _pred_sql = Question(_rewritten_question, _db_id)[0]['inferred_code']
        _gold_sql = c['query']
        _score = EvaluateSQL(_pred_sql, _gold_sql, _db_id)

    c['rewritten_question'] = p['rewritten_question'] = _rewritten_question
    c['pred_sql'] = p['pred_sql'] = _pred_sql
    p['gold_sql'] = _gold_sql
    c['score'] = p['score'] = _score

    pred_idx += len(d)

    

In [ ]:
# Only using the 1st candidate to rewrite 
_avg_1st = sum([d[0]['score'] for d in test_dataset]) / len(test_dataset)

## Std-dev (1st cand only)
_std_1st = np.std([d[0]['score'] for d in test_dataset])

print('avg = {:.4f} (std = {:.4f})'.format(_avg_1st, _std_1st))

In [ ]:
# test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-test-save/{VERSION}.json'

# with open(test_output_path, 'w') as f:
#     json.dump(test_dataset, f, indent=4)

In [ ]:
# Actual (full) evaluation process 

pred_idx = 0

for d in tqdm(test_dataset):
    for c in d:
        p = rewriter_preds[pred_idx]
        _o_idx = c['original_id']
        o = orig_dev_dataset[_o_idx]
        assert ' '.join(c['question_toks']) == p['question'], (' '.join(c['question_toks']), p['question'])
        assert c['gold_question_toks'] == o['question_toks'], (c['gold_question_toks'], o['question_toks'])
        
        pred_idx += 1
        if 'score' in c:
            continue  # already inferred  
        
        _db_id = o['db_id']

        _rewritten_question = ' '.join(p['s2s_prediction'])
        _pred_result = Question(_rewritten_question, _db_id)
        
        _gold_sql = c['query']
        
        if len(_pred_result) == 0:
            print(_db_id, _rewritten_question, '-- no predictiction')
            _pred_sql = ''
            _score = 0
        else:
            _pred_sql = _pred_result[0]['inferred_code']
            _score = EvaluateSQL(_pred_sql, _gold_sql, _db_id)
        
        c['rewritten_question'] = _rewritten_question
        c['pred_sql'] = p['pred_sql'] = _pred_sql
        p['gold_sql'] = _gold_sql
        c['score'] = p['score'] = _score

        


In [ ]:
# Using all the candidates to rewrite 
print(sum([p['score'] for p in rewriter_preds]) / len(rewriter_preds))
print(sum([c['score'] for d in test_dataset for c in d]) / sum([len(d) for d in test_dataset]))

# Only using the 1st candidate to rewrite 
_avg_1st = sum([d[0]['score'] for d in test_dataset]) / len(test_dataset)

## Std-dev (1st cand only)
_std_1st = np.std([d[0]['score'] for d in test_dataset])

print('avg = {:.4f} (std = {:.4f})'.format(_avg_1st, _std_1st))

In [ ]:
# Merge results in a single dataset obj 

test_pred_dataset = []

pred_idx = 0

for d in tqdm(test_dataset):
    _pred_d = []
    
    for c in d:
        p = rewriter_ILM_preds[pred_idx]
        _o_idx = c['original_id']
        o = orig_dev_dataset[_o_idx]
        assert ' '.join(c['question_toks']) == p['question'], (' '.join(c['question_toks']), p['question'])
        assert c['gold_question_toks'] == o['question_toks'], (c['gold_question_toks'], o['question_toks'])
        
        _pred_c = dict()
        
        _pred_c['ASR_question'] = p['question']
        _pred_c['ASR_question_pred_sql'] = c['ratsql_pred_sql']
        
        _pred_c['gold_question'] = c['gold_question']
        # _pred_c['gold_question_pred_sql'] = orig_dev_preds[c['original_id']]
        
        _pred_c['rewritten_question'] = p['s2s_prediction']
        _pred_c['pred_sql'] = p['pred_sql']
        _pred_c['score'] = p['score']
        
        _pred_c['gold_sql'] = c['query']
        
        _pred_d.append(_pred_c)

        pred_idx += 1
    
    test_pred_dataset.append(_pred_d)

len(test_pred_dataset), sum([len(d) for d in test_pred_dataset])

In [ ]:
# test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-test-save/{VERSION}.json'

# with open(test_output_path, 'w') as f:
#     json.dump(test_dataset, f, indent=4)

In [ ]:
test_samples = [c for d in test_dataset for c in d]
for i, c in list(enumerate(test_samples))[8::88]:
    print('-'*30, 'ID = {}'.format(i), '-'*30)
    print('ASR question:\t\t', c['question'])
    print('Rewritten question:\t', c['rewritten_question'])
    print('Gold question:\t\t', c['gold_question'])
    print('Rewritten-q Pred SQL:\t', c['pred_sql'])
#     print('Gold-q Pred SQL:\t', c['gold_question_pred_sql'])
    print('Gold SQL:\t\t', c['query'])
    print('Score:', c['score'])
    

### Evaluating predicted SQL

In [93]:
pred_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Repos/UnifiedSKG/output/server_runs/A-T5_base_prefix_spider_with_cell_value-asr_mixed/predictions_eval_23.95177918190888.json'

with open(pred_dataset_path, 'r') as f:
    pred_dataset = json.load(f)
len(pred_dataset)


5840

In [100]:
pred_dataset[10]

{'prediction': 'select name, country from singer order by age desc',
 'query': 'SELECT name ,  country ,  age FROM singer ORDER BY age DESC',
 'question': 'Shou name country age for all singers ordered by age from the oldest to the youngest.',
 'db_id': 'concert_singer',
 'db_path': '/vault/uskg/data/downloads/extracted/a7a5df4a47a9fc5a13f1b3c5c9ab67d879571a0f9286430e4c3377863a1fb7a1/spider-asr-mixed/database',
 'db_table_names': ['stadium', 'singer', 'concert', 'singer_in_concert'],
 'db_column_names': {'table_id': [-1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   2,
   2,
   2,
   2,
   2,
   3,
   3],
  'column_name': ['*',
   'Stadium_ID',
   'Location',
   'Name',
   'Capacity',
   'Highest',
   'Lowest',
   'Average',
   'Singer_ID',
   'Name',
   'Country',
   'Song_Name',
   'Song_release_year',
   'Age',
   'Is_male',
   'concert_ID',
   'concert_Name',
   'Theme',
   'Stadium_ID',
   'Year',
   'concert_ID',
   'Singer_ID']},
 'db_col

In [101]:
pred_dataset_first_cands = [pred_dataset[0]]

for i in range(1, len(pred_dataset)):
    if pred_dataset[i]['query'] == pred_dataset[i-1]['query']:
        continue
    pred_dataset_first_cands.append(pred_dataset[i])

len(pred_dataset_first_cands)

547

### Rewriter-adaptation Training Data
- Can directly use the existing pipeline, just need to use the tagger-predicted dataset file and ILM output based on that, merge (put in the rewritten text) and get the final dataset file

In [110]:
def _detokenize(toks):
    detokenizer = TreebankWordDetokenizer()
    return detokenizer.detokenize(toks)

In [113]:
def Merge_Tagger_and_ILM_pred(eval_version,
                      rewriter_ILM_pred_path,
                      test_dataset_path,   # like xxx_rewriter+phonemes.json; can be train/dev 
                      # orig_dev_path,
                      test_output_path=None,
                      output_flatten=True, # If true, output a flattened list of samples 
                      ILM_rewrite_func=_Postprocess_rewrite_seq_wrapper,
                      detokenize_func=_detokenize):
    
    '''
    Very similar to Full_evaluate_ILM, process each cand, but not running through NLIDB for SQL prediction 
    
    eval_version: simply for printing results 
    
    Example paths:
    rewriter_ILM_pred_path = '/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/output-{}.json'.format(VERSION)
    test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/test_rewriter.json'
    orig_dev_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/dev.json'
    
    ILM_rewrite_func: Callable, args: (_tags, _rewrite_seq, _question_toks)
    detokenize_func: Callable, args: (toks)
    '''
    
    VERSION = eval_version
    
    with open(rewriter_ILM_pred_path, 'r') as f:
        rewriter_ILM_preds = [json.loads(l) for l in f.readlines()]
    with open(test_dataset_path, 'r') as f:
        test_dataset = json.load(f)
#     with open(orig_dev_path, 'r') as f:
#         orig_dev_dataset = json.load(f)

    # len(rewriter_ILM_preds), len(test_dataset), sum([len(d) for d in test_dataset]), len(orig_dev_dataset)

    pred_idx = 0

    for d in tqdm(test_dataset, desc=f'VERSION {VERSION}'):
        for c in d:
            p = rewriter_ILM_preds[pred_idx]
            _o_idx = c['original_id']
            # o = orig_dev_dataset[_o_idx]
            assert ' '.join(c['question_toks']) == p['question'], (' '.join(c['question_toks']), p['question'])
            # assert c['gold_question_toks'] == o['question_toks'], (c['gold_question_toks'], o['question_toks'])

            # Debug 
            # assert c['rewriter_tags'] == p['rewriter_tags'][:len(c['rewriter_tags'])], f"{c['rewriter_tags']}\n{p['rewriter_tags']}\nShould raise"

            _tags = p['rewriter_tags']
            _rewrite_seq = p['rewrite_seq_prediction']
            _question_toks = c['question_toks']

            # _rewritten_question_toks = Postprocess_rewrite_seq(_tags, _rewrite_seq, _question_toks)
            _rewritten_question_toks = ILM_rewrite_func(c, p)
            # _rewritten_question = ' '.join(_rewritten_question_toks)
            _rewritten_question = detokenize_func(_rewritten_question_toks)

            # Save the taggerILM raw outputs, for later aggregation 
            c['pred_tags'] = p['rewriter_tags']
            c['pred_ILM'] = p['rewrite_seq_prediction']
            c['pred_ILM_cands'] = p['rewrite_seq_prediction_cands']

            # Save prediction results 
            # keep the asr question (just in case)
            c['asr_question'] = c['question']
            c['asr_question_toks'] = c['question_toks']
            # Directly write to key "question" and "question_toks" to help later training 
            # c['rewritten_question'] = p['rewritten_question'] = _rewritten_question
            c['question'] = _rewritten_question
            c['question_toks'] = _rewritten_question_toks

            pred_idx += 1

    if test_output_path is not None:
        if output_flatten:
            output_struct = [c for d in test_dataset for c in d]
        else:
            output_struct = test_dataset
        with open(test_output_path, 'w') as f:
            json.dump(output_struct, f, indent=2)
    

In [ ]:
VERSION = 'dev-2.12.1.0t-2.29.0.0i'
rewriter_ILM_pred_path = '/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/output-{}.json'.format(VERSION)
test_dataset_path = '/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/dev/dev_rewriter.json'
# test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/ratsql-test-save/{VERSION}.json'
test_output_path = f'/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/aside/retrain-rewritten/{VERSION}.json'

Merge_Tagger_and_ILM_pred(eval_version=VERSION,
                  rewriter_ILM_pred_path=rewriter_ILM_pred_path,
                  test_dataset_path=test_dataset_path,
                  test_output_path=test_output_path)



### Format eval result files

In [397]:
# FNAME_LIST = [f'eval-humantest-yshao-1.15.2.{v}' for v in range(4)]
# FNAME_LIST = [f'eval-2.12.1.{v}t-2.33.9.{v}i' for v in range(4)]
# FNAME_LIST = [f'eval-dev-2.12.1.{v}t-2.33.8.{v}i' for v in range(4)]
FNAME_LIST = [f'eval-2.12.1.{v}t-2.35.1.{v}i' for v in range(4)]
# backend = 'uskg'
backends = ['ratsql', 'uskg', 'uskg-large']

for backend in backends:
    print(backend)
    for FNAME in FNAME_LIST:
        res_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/{backend}-test-save/{FNAME}.json'
        with open(res_path, 'r') as f:
            d = json.load(f)
    #     print(f"{d['avg_exact']:.4f}\t{d['avg']:.4f}\t{d['avg_exec']:.4f}\t{d['BLEU']:.4f}")
        print(f"{d['avg_exact']:.4f}\t{d['avg']:.4f}\t{d['avg_exec']:.4f}\t{d['BLEU']:.4f}\t{d['WER']:.4f}")
    print()

ratsql
0.5229	0.7673	0.3821	0.8594	0.0754
0.5247	0.7713	0.3803	0.8807	0.0633
0.5192	0.7622	0.3638	0.8727	0.0680
0.5411	0.7819	0.3821	0.8771	0.0690

uskg
0.4863	0.6289	0.4845	0.8594	0.0754
0.4954	0.6270	0.4954	0.8807	0.0633
0.4826	0.6244	0.4698	0.8727	0.0680
0.5064	0.6377	0.5009	0.8771	0.0690

uskg-large
0.5558	0.7229	0.5539	0.8594	0.0754
0.5960	0.7455	0.5850	0.8807	0.0633
0.5777	0.7253	0.5576	0.8727	0.0680
0.5850	0.7398	0.5759	0.8771	0.0690



In [313]:
POS_LIST = ["PUNCT", "NUM", "VERB", "PRON", "ADP", "NOUN", "AUX", "DET",
    "SCONJ", "PART", "ADJ", "CCONJ", "ADV", "PROPN"]

# FNAME_LIST = [f'eval-1.15.1.{v}' for v in range(4)]
# FNAME_LIST = [f'eval-2.12.1.1t-2.31.0.1i-freeze={pos}' for pos in POS_LIST]
# FNAME_LIST = [f'eval-humantest-yshao-1.16.0.{v}' for v in range(4)]
# FNAME_LIST = [f'eval-humantest-yshao-2.31.0.{v}i-oracle-tags' for v in range(4)]
backend = 'ratsql'

for _pos in POS_LIST:
    _run_name = f'eval-2.12.1.1t-2.33.5.1i-freeze={_pos}'
    res_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/outputs/{backend}-test-save/{_run_name}.json'
    with open(res_path, 'r') as f:
        d = json.load(f)
    print(f"{_pos}\t{d['avg_exact']:.4f}")
#     print(f"{d['avg_exact']:.4f}\t{d['avg']:.4f}\t{d['avg_exec']:.4f}\t{d['BLEU']:.4f}\t{d['WER']:.4f}")

PUNCT	0.5210
NUM	0.5320
VERB	0.5375
PRON	0.5283
ADP	0.5283
NOUN	0.5046
AUX	0.5393
DET	0.5375
SCONJ	0.5411
PART	0.5411
ADJ	0.5430
CCONJ	0.5411
ADV	0.5411
PROPN	0.5338


### Other info

In [343]:
def _duration_to_hours(dur_str):
    if 'day' not in dur_str:
        _nday = 0
        _hour_str = dur_str
    else:
        _day_str, _hour_str = dur_str.split(' day, ')
        _nday = int(_day_str)
        
    _time = datetime.datetime.strptime(_hour_str.split('.')[0], "%H:%M:%S")
    _hours = _nday * 24 + _time.hour + _time.minute / 60 + _time.second / 3600
    
    return _hours

In [345]:
_duration_to_hours("19:44:49.731672")

19.746944444444445

In [394]:
# VERSION_LIST = [f'2.12.1.{v}t' for v in range(4)]
VERSION_LIST = [f'2.35.1.{v}i' for v in range(4)]
# VERSION_LIST = [f'1.15.2.{v}' for v in range(4)]

_hours_list = []
_patience = 20

for ver in VERSION_LIST:
    print(ver)
    res_path = f'/Users/mac/Desktop/syt/Deep-Learning/Projects-M/SpeakQL/SpeakQL/Allennlp_models/runs/{ver}/metrics.json'
    with open(res_path, 'r') as f:
        d = json.load(f)
    
    _dur_str = d['training_duration']
    _hours = _duration_to_hours(_dur_str)
    
    ## Unify the patience 
    _epochs = d['training_epochs']
    _best_ep = d['best_epoch']
    print(_epochs, '->', _best_ep + _patience - 1)
    
    _hours = _hours / _epochs * (_best_ep + _patience - 1)
    
    _hours_list.append(_hours)

print('\t'.join([f"{_h:.4f}" for _h in _hours_list]))
    

2.35.1.0i
39 -> 39
2.35.1.1i
29 -> 29
2.35.1.2i
26 -> 26
2.35.1.3i
30 -> 30
6.2694	4.6453	4.2556	4.8972


### Temp

In [ ]:
rewriter_ILM_preds[0].keys()

In [ ]:
test_dataset[0][0].keys()

In [ ]:
orig_dev_dataset[0].keys()

In [ ]:
# Rewrite_seq postprocessing, to get the rewritten question 

_idx = 154

_tags = rewriter_ILM_preds[_idx]['tags_prediction']
_rewrite_seq = rewriter_ILM_preds[_idx]['rewrite_seq_prediction']
_question_toks = rewriter_ILM_preds[_idx]['question'].split(' ')
_tags, _rewrite_seq, _question_toks

postprocess_rewrite_seq(_tags, _rewrite_seq, _question_toks)

In [ ]:
db = 'concert_singer'
g_str = 'SELECT count(*) FROM singer'
p_str = "SELECT Count(DISTINCT singer.Name) FROM singer WHERE singer.Name = 'terminal'"

db, p_str, g_str, EvaluateSQL(p_str, g_str, db)

In [107]:
with open('/Users/mac/Desktop/syt/Deep-Learning/Dataset/spider/my/train/train_reranker.json') as f:
    _train = json.load(f)
len(_train), sum([len(d) for d in _train])

(7000, 41112)

In [108]:
del _train

In [109]:
%load_ext memory_profiler

In [110]:
%memit

peak memory: 623.44 MiB, increment: -2.01 MiB


In [223]:
_d = nlp("While the id between kyle and jetblue airways are good")
list(zip([t.text for t in _d], [t.pos_ for t in _d]))

[('While', 'SCONJ'),
 ('the', 'DET'),
 ('i', 'PRON'),
 ('d', 'NOUN'),
 ('between', 'ADP'),
 ('kyle', 'PROPN'),
 ('and', 'CCONJ'),
 ('jetblue', 'PROPN'),
 ('airways', 'NOUN'),
 ('are', 'AUX'),
 ('good', 'ADJ')]

In [271]:
[random.randint(1, 1e12) for _ in range(10)]

[540223612631,
 178720741399,
 109777393677,
 817735554319,
 340641764082,
 582783668677,
 739916569103,
 548295624987,
 793475247955,
 140832154574]